# Week 1

In [ ]:
#~source: https: //en.wikipedia.org/wiki/Gradient_descent
# code source: https://en.wikipedia.org/w/index.php?title=Gradient_descent&oldid=966271567

# 初始值
next_x = 6# We start the search at x = 6
# 步长系数（学习率）
gamma = 0.01# Step size multiplier
# 提前停止（系数变化小于此值时停止）
precision = 0.00001# Desired precision of result
# 最大迭代次数
max_iters = 10000# Maximum number of iterations

# Derivative
#function
#求导
def df(x):
  return 4 * x ** 3 - 9 * x ** 2

# 迭代
for i in range(max_iters):
    current_x = next_x
    
    #梯度下降
    next_x = current_x - gamma * df(current_x)
    print(i, next_x, df(current_x))

    # 提前停止的判定，计算系数变化了多少，小于阈值后提前停止
    step = next_x - current_x
    if abs(step) <= precision:
        break

print("Minimum at ", next_x)

# The output for the above will be something like 
# "Minimum at 2.2499646074278457"

In [ ]:
# Source: https://github.com/mattnedrich/GradientDescentExample
# y = mx + b
# m is slope, b is y-intercept
# 计算均方误差 MSE
def compute_error_for_line_given_points(b, m, points):
    totalError = 0
    for i in range(0, len(points)):
        x = points[i, 0]
        y = points[i, 1]
        # 实际的y减去 mx+b 的 y
        totalError += (y - (m * x + b)) ** 2
    return totalError / float(len(points))

In [ ]:
# Source: https://github.com/mattnedrich/GradientDescentExample

# 梯度下降用于线性回归
def step_gradient(b_current, m_current, points, learningRate):
    b_gradient = 0
    m_gradient = 0
    N = float(len(points))
    for i in range(0, len(points)):
        x = points[i, 0]
        y = points[i, 1]
        
        # 负梯度计算 （Loss Function 对 y_pre的导数，乘以 y_pred 对 m和b的导数
        # 除以N是因为有N个样本，相当于取平均值
        b_gradient += -(2/N) * (y - ((m_current * x) + b_current))
        m_gradient += -(2/N) * x * (y - ((m_current * x) + b_current))
    new_b = b_current - (learningRate * b_gradient)
    new_m = m_current - (learningRate * m_gradient)
    return [new_b, new_m]

In [ ]:
# 执行梯度下降的函数
def gradient_descent_runner(points, starting_b, starting_m, learning_rate, num_iterations):
    b = starting_b
    m = starting_m
    for i in range(num_iterations):
        b, m = step_gradient(b, m, array(points), learning_rate)
    return [b, m]

In [ ]:
# 用上面的代码合到一起，跑起来
def run():
    points = genfromtxt("data_linearreg.csv", delimiter=",")
    learning_rate = 0.0001
    initial_b = 0 # initial y-intercept guess
    initial_m = 0 # initial slope guess
    num_iterations = 1000
    print ("Starting gradient descent at b = {0}, m = {1}, error = {2}".format(initial_b, initial_m, compute_error_for_line_given_points(initial_b, initial_m, points)))
    print ("Running...")
    [b, m] = gradient_descent_runner(points, initial_b, initial_m, learning_rate, num_iterations)
    print ("After {0} iterations b = {1}, m = {2}, error = {3}".format(num_iterations, b, m, compute_error_for_line_given_points(b, m, points)) )

In [ ]:
#source: https://stackoverflow.com/questions/3949226/calculating-pearson-correlation-and-significance-in-python

import math
import numpy as np
from random import random

# 计算 R Score
def pcc(X, Y):
   ''' Compute Pearson Correlation Coefficient. '''
   # Normalise X and Y
   X -= X.mean(0)
   Y -= Y.mean(0)
   # Standardise X and Y
   X /= X.std(0)
   Y /= Y.std(0)
   # Compute mean product
   return np.mean(X*Y)
 
def average(x):
    assert len(x) > 0
    return float(sum(x)) / len(x)

def pearson_def(x, y):
    assert len(x) == len(y)
    n = len(x)
    assert n > 0
    avg_x = average(x)
    avg_y = average(y)
    diffprod = 0
    xdiff2 = 0
    ydiff2 = 0
    for idx in range(n):
        xdiff = x[idx] - avg_x
        ydiff = y[idx] - avg_y
        diffprod += xdiff * ydiff
        xdiff2 += xdiff * xdiff
        ydiff2 += ydiff * ydiff

    # 协方差 除以 两个标准差的乘积
    return diffprod / math.sqrt(xdiff2 * ydiff2)

#main

# Using it on a random example

X = np.array([random() for x in range(100)])
Y = np.array([random() for x in range(100)])


# 两种等效的写法
pcof = pcc(X, Y)
print(pcof, ' is pcof')
 
pcoftwo = pearson_def(X, Y)
print(pcoftwo, ' is pcof second version')

# Week 2

In [ ]:
#Source: https://machinelearningmastery.com/implement-logistic-regression-stochastic-gradient-descent-scratch-python/

from math import exp

# Make a prediction with coefficients
# 预测输出的概率 
# 先计算 y_hat， 再使用 sigmoid函数转化为概率
def predict(row, coefficients):
	yhat = coefficients[0]
	for i in range(len(row)-1):
		yhat += coefficients[i + 1] * row[i]
	return 1.0 / (1.0 + exp(-yhat))

# Estimate logistic regression coefficients using stochastic gradient descent
# 梯度下降 更新参数 (权重 w 和 b) 
# 虽然标注了SGD，但这不是一个随机梯度下降，而是使用 SSE作为损失函数的GD： （全量的）批量梯度下降（如果使用MSE，求导时也要取均值，乘以1/N）
def coefficients_sgd(train, l_rate, n_epoch):
    # 初始化系数
	coef = [0.0 for i in range(len(train[0]))]
    # 迭代过程
	for epoch in range(n_epoch):
		sum_error = 0
        # 循环 遍历每一个sample
		for row in train:
            # 计算 预测的prop
			yhat = predict(row, coef)
            # 计算 伪残差
			error = row[-1] - yhat
            # 计算 伪残差平方和，类似MSE
			sum_error += error**2
            # 在原有的 系数基础上，加上 负梯度乘以学习率乘以 sigmoid 的导数
            # 这里是在更新 b(bias) wx+b的b
			coef[0] = coef[0] + l_rate * error * yhat * (1.0 - yhat)
			#print(row, yhat, error, sum_error)
            # 这里是更新 w
			for i in range(len(row)-1):
				coef[i + 1] = coef[i + 1] + l_rate * error * yhat * (1.0 - yhat) * row[i]
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
	return coef

# Calculate coefficients
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
l_rate = 0.3
n_epoch = 10
coef = coefficients_sgd(dataset, l_rate, n_epoch)
print(coef)

In [ ]:
 
 # by R. Chandra
 #Source: https://github.com/rohitash-chandra/logistic_regression

from math import exp
import numpy as np
import random

SIGMOID = 1
STEP = 2
LINEAR = 3

 
random.seed()

class logistic_regression:

    # num_epocs 迭代次数
    # train_data 训练集
    # test_data 测试集
    # num_features 特征数量
    # learn_rate 学习率
	def __init__(self, num_epocs, train_data, test_data, num_features, learn_rate):
		self.train_data = train_data
		self.test_data = test_data 
		self.num_features = num_features
        # 输出数据维度，列数减去输入特征数量
		self.num_outputs = self.train_data.shape[1] - num_features
        # 样本量
		self.num_train = self.train_data.shape[0]
		self.w = np.random.uniform(-0.5, 0.5, num_features)  # in case one output class 
		self.b = np.random.uniform(-0.5, 0.5, self.num_outputs) 
		self.learn_rate = learn_rate
		self.max_epoch = num_epocs
		self.use_activation = SIGMOID #SIGMOID # 1 is  sigmoid , 2 is step, 3 is linear 
        # 用于记录偏导数
		self.out_delta = np.zeros(self.num_outputs)

		print(self.w, ' self.w init') 
		print(self.b, ' self.b init') 
		print(self.out_delta, ' outdel init')


    # 根据非激活值计算激活值
	def activation_func(self,z_vec):
		if self.use_activation == SIGMOID:
			y =  1 / (1 + np.exp(z_vec)) # sigmoid/logistic
		elif self.use_activation == STEP:
			y = (z_vec > 0).astype(int) # if greater than 0, use 1, else 0
			#https://stackoverflow.com/questions/32726701/convert-real-valued-numpy-array-to-binary-array-by-sign
		else:
			y = z_vec
		return y
 
    # 计算y的输出值
	def predict(self, x_vec ): 
		z_vec = x_vec.dot(self.w) - self.b 
		output = self.activation_func(z_vec) # Output  
		return output
	
	# 计算梯度 （已修正）
    # 预测值减实际值：梯度， 实际值减预测值：负梯度
	def gradient(self, x_vec, output, actual):   
		if self.use_activation == SIGMOID :
			out_delta =   (output - actual)*(output*(1-output)) 
		else: # for linear and step function  
			out_delta =   (output - actual) 
		return out_delta

	# 更新参数：这里有问题，正确写法是改成-=
	def update(self, x_vec, output, actual):      
		self.w+= self.learn_rate *( x_vec *  self.out_delta)
		self.b+=  (1 * self.learn_rate * self.out_delta)
 
	# 计算 SSE
    # 对于分类问题，这里的计算 将概率与实际值之差 视为伪残差
	def squared_error(self, prediction, actual):
		return  np.sum(np.square(prediction - actual))/prediction.shape[0]# to cater more in one output/class
 

	# 评估模型
	def test_model(self, data, tolerance):  

		num_instances = data.shape[0]

		class_perf = 0
		sum_sqer = 0   
        
        #循环遍历每一个样本
		for s in range(0, num_instances):	

			input_instance  =  self.train_data[s,0:self.num_features] 
			actual  = self.train_data[s,self.num_features:]  
			prediction = self.predict(input_instance)
			sum_sqer += self.squared_error(prediction, actual)

			# 设置分类概率阈值
			pred_binary = np.where(prediction > (1 - tolerance), 1, 0)

			print(s, actual, prediction, pred_binary, sum_sqer, ' s, actual, prediction, sum_sqer')

 
			# 预测正确 +1
			if( (actual==pred_binary).all()):
				class_perf =  class_perf +1   

		rmse = np.sqrt(sum_sqer/num_instances)

		percentage_correct = float(class_perf/num_instances) * 100 

		print(percentage_correct, rmse,  ' class_perf, rmse') 
		# note RMSE is not a good measure for multi-class probs

		return ( rmse, percentage_correct)





 
	def SGD(self):   
		
			epoch = 0 
			shuffle = True

			while  epoch < self.max_epoch:
				sum_sqer = 0
				for s in range(0, self.num_train): 

					if shuffle ==True:
						i = random.randint(0, self.num_train-1)

					input_instance  =  self.train_data[i,0:self.num_features]  
					actual  = self.train_data[i,self.num_features:]  
					prediction = self.predict(input_instance) 
					sum_sqer += self.squared_error(prediction, actual)
					self.out_delta = self.gradient( input_instance, prediction, actual)    # major difference when compared to GD
					#print(input_instance, prediction, actual, s, sum_sqer)
                    
                    # 使用梯度，更新参数
					self.update(input_instance, prediction, actual)

			
				print(epoch, sum_sqer, self.w, self.b)
				epoch=epoch+1  

			rmse_train, train_perc = self.test_model(self.train_data, 0.3) 
			rmse_test =0
			test_perc =0
			#rmse_test, test_perc = self.test_model(self.test_data, 0.3)
  
			return (train_perc, test_perc, rmse_train, rmse_test) 
				
	# 梯度下降
	def GD(self):   
		
			epoch = 0 
			while  epoch < self.max_epoch:
				sum_sqer = 0
				for s in range(0, self.num_train): 
					input_instance  =  self.train_data[s,0:self.num_features]  
					actual  = self.train_data[s,self.num_features:]   
					prediction = self.predict(input_instance) 
					sum_sqer += self.squared_error(prediction, actual) 
					self.out_delta+= self.gradient( input_instance, prediction, actual)    # this is major difference when compared with SGD

					#print(input_instance, prediction, actual, s, sum_sqer)
                
					# 使用梯度，更新参数
					self.update(input_instance, prediction, actual)

			
				print(epoch, sum_sqer, self.w, self.b)
				epoch=epoch+1  

			rmse_train, train_perc = self.test_model(self.train_data, 0.3) 
			rmse_test =0
			test_perc =0
			#rmse_test, test_perc = self.test_model(self.test_data, 0.3)
  
			return (train_perc, test_perc, rmse_train, rmse_test) 
				
	
 

#------------------------------------------------------------------
#MAIN



def main(): 

	random.seed()
	 

	 
	dataset = [[2.7810836,2.550537003,0],
		[1.465489372,2.362125076,0],
		[3.396561688,4.400293529,0],
		[1.38807019,1.850220317,0],
		[3.06407232,3.005305973,0],
		[7.627531214,2.759262235,1],
		[5.332441248,2.088626775,1],
		[6.922596716,1.77106367,1],
		[8.675418651,-0.242068655,1],
		[7.673756466,3.508563011,1]]


	train_data = np.asarray(dataset) # convert list data to numpy
	test_data = train_data

	 

	learn_rate = 0.3
	num_features = 2
	num_epocs = 20

	print(train_data)
	 

	lreg = logistic_regression(num_epocs, train_data, test_data, num_features, learn_rate)
	(train_perc, test_perc, rmse_train, rmse_test) = lreg.SGD()
	(train_perc, test_perc, rmse_train, rmse_test) = lreg.GD() 
	 

	#-------------------------------
	#xor data


	xor_dataset= [[0,0,0],
		[0,1,1],
		[1,0,1],
		[1,1,0] ]

	xor_data = np.asarray(xor_dataset) # convert list data to numpy



	num_epocs = 20
	learn_rate = 0.9
	num_features = 2

	lreg = logistic_regression(num_epocs, xor_data, xor_data, num_features, learn_rate)
	(train_perc, test_perc, rmse_train, rmse_test) = lreg.SGD()
	(train_perc, test_perc, rmse_train, rmse_test) = lreg.GD() 


if __name__ == "__main__": main()

In [ ]:
# LogLoss计算

import numpy as np
# 对数损失计算函数
def loss(h, y):
    return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()

# 下面是例子：用这个函数做了两个计算
h= np.random.rand(5)
print(h, ' h')
y= np.random.rand(5)
print(y, ' y')
  
log_loss = loss(h, y) # case of regression or prediction problem
print(log_loss)

y_ = np.ones(5)
print(y_, ' y_')
log_loss = loss(h, y_) # case of classification problem (assume class one 1s)
print(log_loss)

# Week 3

In [ ]:
#https://gist.githubusercontent.com/Thomascountz/77670d1fd621364bc41a7094563a7b9c/raw/39091aea4beb7bd33e7c492bdd8b1c3a344ca594/perceptron.py
# Copyright (c) 2018 Thomas Countz

import numpy as np
'''
这段代码实现了一个简单的感知器模型，用来训练和预测逻辑操作。以下是它的主要功能：

定义了一个感知器类 (Perceptron)：

初始化方法 (__init__) 接收输入特征的数量、阈值（用于循环次数）以及学习率。
weights 表示权重向量，初始时全为零。
predict 方法：

计算输入数据的加权和（包括偏置），并基于阈值判断输出（激活函数）。
输出为1或0，表示激活或不激活。
train 方法：

使用给定的训练数据和标签来训练模型。
对于每一个训练样本，预测输出值，然后根据真实标签进行权重的更新。
使用感知器模型训练一个简单的逻辑门：

training_inputs 是逻辑门的输入，例如 [1, 1], [1, 0], [0, 1], [0, 0]。
labels 是目标输出，例如 [1, 0, 0, 0] 表示一个与门 (AND gate) 的输出。
创建感知器对象并训练模型，然后使用训练好的模型进行预测，打印出结果。

简单来说，这段代码实现了一个基于感知器的逻辑门分类器（例如 AND 或 XOR），通过训练感知器来对输入做出预测。
'''

class Perceptron(object):

    def __init__(self, no_of_inputs, threshold, learning_rate):
        self.threshold = threshold
        self.learning_rate = learning_rate
        self.weights = np.zeros(no_of_inputs + 1)
           
    def predict(self, inputs):
        summation = np.dot(inputs, self.weights[1:]) + self.weights[0]
        if summation > 0:
          activation = 1
        else:
          activation = 0            
        return activation

    def train(self, training_inputs, labels):
        for _ in range(self.threshold):

            for inputs, label in zip(training_inputs, labels):
                prediction = self.predict(inputs)
                #print(prediction, label)
                self.weights[1:] += self.learning_rate * (label - prediction) * inputs
                self.weights[0] += self.learning_rate * (label - prediction) #bias


# set the dataset inputs
# AND gate 

training_inputs = []
training_inputs.append(np.array([1, 1]))
training_inputs.append(np.array([1, 0]))
training_inputs.append(np.array([0, 1]))
training_inputs.append(np.array([0, 0]))

# set the dataset class labels
labels = np.array([1, 0, 0, 0])# AND
#labels = np.array([1, 0, 0, 1]) #XOR
#labels = np.array([1, 1, 1, 0]) # OR

no_of_inputs = 2
threshold=100
learning_rate=0.01

# set the class and train
perceptron = Perceptron(no_of_inputs, threshold, learning_rate)
perceptron.train(training_inputs, labels)

# now test trained percepton
inputs = np.array([1, 1])
output = perceptron.predict(inputs) 
print(output, ' for input [1, 1]' ) 

inputs = np.array([0, 1])
output = perceptron.predict(inputs) 
print(output, ' for input [0, 1]' ) 

In [ ]:

# Topo = [3,4, 2]

'''这段代码定义了一个简单的多层前馈神经网络（MLP）类 Network，实现了神经网络的初始化过程，包括网络的拓扑结构、权重和偏置的随机初始化等。以下是代码主要部分的功能：

类 Network 的初始化方法 (__init__):

Topo：网络的拓扑结构（例如 [3, 4, 2] 表示网络有 3 个输入节点，4 个隐藏层节点，和 2 个输出节点）。
Train 和 Test：训练数据和测试数据。
MaxTime：最大训练轮数（epocs）。
Samples：样本数量。
MinPer：最小的性能要求，用于终止训练的条件之一。
learnRate：学习率，用于权重更新时的步长控制。
初始化网络的权重和偏置:

W1：输入层到隐藏层的权重矩阵，维度为 (Topo[0], Topo[1])，随机初始化在 [-0.5, 0.5] 之间。
B1：隐藏层的偏置，维度为 (1, Topo[1])，随机初始化在 [-0.5, 0.5] 之间。
W2：隐藏层到输出层的权重矩阵，维度为 (Topo[1], Topo[2])，随机初始化在 [-0.5, 0.5] 之间。
B2：输出层的偏置，维度为 (1, Topo[2])，随机初始化在 [-0.5, 0.5] 之间。
BestW1、BestB1、BestW2、BestB2：用于保存最优的权重和偏置。
隐藏层和输出层初始化:

hidout：存储隐藏层的输出，初始化为全零。
out：存储输出层的输出，初始化为全零。
hid_delta 和 out_delta：分别用于存储隐藏层和输出层的误差项，初始化为全零。
随机权重和偏置的初始化:

使用 np.random.uniform(-0.5, 0.5, ...) 来随机初始化权重和偏置，使得它们的初始值在 [-0.5, 0.5] 之间。这样有助于避免所有权重都为同一个值的问题，提高模型的收敛性。'''

import matplotlib.pyplot as plt
import numpy as np 
import random
import time

class Network:

	def __init__(self, Topo, Train, Test, MaxTime, Samples, MinPer, learnRate): 
		self.Top  = Topo  # NN topology [input, hidden, output]
		self.Max = MaxTime # max epocs
		self.TrainData = Train
		self.TestData = Test
		self.NumSamples = Samples

		self.learn_rate  = learnRate
 

		self.minPerf = MinPer
		
		#initialize weights ( W1 W2 ) and bias ( b1 b2 ) of the network
		np.random.seed() 
		self.W1 = np.random.uniform(-0.5, 0.5, (self.Top[0] , self.Top[1]))  
		#print(self.W1,  ' self.W1')
		self.B1 = np.random.uniform(-0.5,0.5, (1, self.Top[1])  ) # bias first layer
		#print(self.B1, ' self.B1')
		self.BestB1 = self.B1
		self.BestW1 = self.W1 
		self.W2 = np.random.uniform(-0.5, 0.5, (self.Top[1] , self.Top[2]))   
		self.B2 = np.random.uniform(-0.5,0.5, (1,self.Top[2]))  # bias second layer
		self.BestB2 = self.B2
		self.BestW2 = self.W2 
		self.hidout = np.zeros(self.Top[1] ) # output of first hidden layer
		self.out = np.zeros(self.Top[2]) #  output last layer

		self.hid_delta = np.zeros(self.Top[1] ) # output of first hidden layer
		self.out_delta = np.zeros(self.Top[2]) #  output last layer

# Week 4

In [ ]:
#Source: https://www.python-course.eu/neural_networks_with_scikit.php
'''
这段代码实现了一个神经网络的反向传播算法，用于计算误差并更新网络的权重和偏置。以下是具体解释：

BackwardPass 方法：

参数：
input_vec：输入向量，表示当前样本输入到神经网络的特征。
desired：目标值，表示当前样本的真实标签。
输出层误差计算 (out_delta)：

out_delta = (desired - self.out) * (self.out * (1 - self.out))：
desired - self.out：计算输出误差，即目标输出与神经网络当前预测值的差异。
self.out * (1 - self.out)：对输出层进行激活函数的导数计算，这里假设使用的是 Sigmoid 激活函数，其导数为 f(x) * (1 - f(x))。
out_delta 表示输出层的误差项，反映了神经网络的输出与期望输出之间的偏差。
隐藏层误差计算 (hid_delta)：

hid_delta = out_delta.dot(self.W2.T) * (self.hidout * (1 - self.hidout))：
使用 out_delta 和隐藏层到输出层的权重矩阵 self.W2 来计算隐藏层的误差项。
out_delta.dot(self.W2.T)：将输出层误差反向传播到隐藏层，dot 表示矩阵乘法。
self.hidout * (1 - self.hidout)：对隐藏层的激活值进行激活函数的导数计算，这里同样假设使用的是 Sigmoid 激活函数。
hid_delta 表示隐藏层的误差项，反映了隐藏层节点对最终输出误差的贡献。
更新权重和偏置：

更新输出层权重和偏置：
self.W2 += self.hidout.T.dot(out_delta) * self.learn_rate：使用隐藏层的输出和输出层误差项来更新 W2，即隐藏层到输出层的权重。
self.B2 += (-1 * self.learn_rate * out_delta)：更新输出层的偏置 B2，调整方向与学习率有关。
更新隐藏层权重和偏置：
self.W1 += input_vec.T.dot(hid_delta) * self.learn_rate：使用输入向量和隐藏层误差项来更新 W1，即输入层到隐藏层的权重。
self.B1 += (-1 * self.learn_rate * hid_delta)：更新隐藏层的偏置 B1。
学习率的作用：

学习率 self.learn_rate 控制每次更新的步长大小，避免过大的权重更新导致模型发散。
'''

#FNN Version Two
	def BackwardPass(self, input_vec, desired):   
		out_delta =   (desired - self.out)*(self.out*(1-self.out))  
		hid_delta = out_delta.dot(self.W2.T) * (self.hidout * (1-self.hidout)) #https://www.tutorialspoint.com/numpy/numpy_dot.htm  https://www.geeksforgeeks.org/numpy-dot-python/
  
		self.W2+= self.hidout.T.dot(out_delta) * self.learn_rate
		self.B2+=  (-1 * self.learn_rate * out_delta)

		self.W1 += (input_vec.T.dot(hid_delta) * self.learn_rate) 
		self.B1+=  (-1 * self.learn_rate * hid_delta) 

In [ ]:
'''
这段代码定义了一个函数 BackwardPass，用于实现一个简单神经网络的反向传播（Backpropagation），它是训练神经网络的关键步骤。函数通过调整权重和偏置（weights and biases）来最小化损失。下面是对代码的详细解释：

函数定义和输入参数：

self：引用类实例，用于访问对象的属性。
input_vec：输入向量，即用于前向传播的输入数据。
desired：期望的输出（真实标签），用于计算误差。
输出层误差计算 (out_delta)：

out_delta = (desired - self.out) * (self.out * (1 - self.out))：
这里 (desired - self.out) 表示输出层的误差。
self.out * (1 - self.out) 是 Sigmoid 激活函数的导数。
out_delta 表示输出层的误差项，用于计算梯度。
隐藏层误差计算 (hid_delta)：

hid_delta = out_delta.dot(self.W2.T) * (self.hidout * (1 - self.hidout))：
out_delta.dot(self.W2.T) 计算输出层误差对隐藏层的影响（反向传播）。
self.hidout * (1 - self.hidout) 是隐藏层激活函数（Sigmoid）的导数。
hid_delta 表示隐藏层的误差项。
更新权重和偏置（根据是否使用动量）：

如果 self.vanilla == True，即没有动量：

更新输出层权重 W2：self.W2 += self.hidout.T.dot(out_delta) * self.learn_rate
使用隐藏层输出与输出层误差计算梯度，再乘以学习率 learn_rate。
更新输出层偏置 B2：self.B2 += (-1 * self.learn_rate * out_delta)。
更新隐藏层权重 W1：self.W1 += input_vec.T.dot(hid_delta) * self.learn_rate。
更新隐藏层偏置 B1：self.B1 += (-1 * self.learn_rate * hid_delta)。
如果使用动量：

使用 self.momenRate 和之前的权重、偏置值来加速收敛并避免震荡。
使用动量来更新权重和偏置时：
先将当前权重和偏置保存在 v2, v1, b2, b1 中。
更新权重时，不仅考虑当前的梯度，还加入了一部分的动量（即之前的权重变化）。
'''

def BackwardPass(self, input_vec, desired):   
		out_delta =   (desired - self.out)*(self.out*(1-self.out))  
		hid_delta = out_delta.dot(self.W2.T) * (self.hidout * (1-self.hidout)) 
		#https://www.tutorialspoint.com/numpy/numpy_dot.htm  https://www.geeksforgeeks.org/numpy-dot-python/
  
		if self.vanilla == True: #no momentum 
			self.W2+= self.hidout.T.dot(out_delta) * self.learn_rate
			self.B2+=  (-1 * self.learn_rate * out_delta)

			self.W1 += (input_vec.T.dot(hid_delta) * self.learn_rate) 
			self.B1+=  (-1 * self.learn_rate * hid_delta) 
		else: # use momentum
			v2 = self.W2.copy() #save previous weights http://cs231n.github.io/neural-networks-3/#sgd
			v1 = self.W1.copy()
			b2 = self.B2.copy()
			b1 = self.B1.copy()

			self.W2+= ( v2 *self.momenRate) + (self.hidout.T.dot(out_delta) * self.learn_rate)       # velocity update
			self.W1 += ( v1 *self.momenRate) + (input_vec.T.dot(hid_delta) * self.learn_rate)   
			self.B2+= ( b2 *self.momenRate) + (-1 * self.learn_rate * out_delta)       # velocity update
			self.B1 += ( b1 *self.momenRate) + (-1 * self.learn_rate * hid_delta)   

In [ ]:

#useStocastic = True # False for vanilla BP with SGD (no shuffle of data). # True for BP with SGD (shuffle of data at every epoch) 

#updateStyle = False # True for Vanilla SGD, False for momentum SGD

def BP_GD(self, trainTolerance):  
		Input = np.zeros((1, self.Top[0])) # Temp hold input
		Desired = np.zeros((1, self.Top[2])) 

		#minibatchsize = int(0.1* self.TrainData.shape[0]) # Choose a mini-batch size for SGD - optional exercise to implement this

		Er = [] 
		epoch = 0
		bestRMSE= 10000 # Assign a large number in begining to maintain best (lowest RMSE)
		bestTrain = 0
		while  epoch < self.Max and bestTrain < self.minPerf :
			sse = 0

			for s in range(0, self.TrainData.shape[0]):
				if(self.stocasticGD==True):   
					pat = random.randint(0, self.TrainData.shape[0]-1) 
					# Data shuffle in SGD: https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.random.randint.html
				else:
					pat = s # no data shuffle in SGD
		
				Input[:]  =  self.TrainData[pat,0:self.Top[0]]  
				Desired[:]  = self.TrainData[pat,self.Top[0]:]  

				self.ForwardPass(Input)  
				self.BackwardPass(Input ,Desired)
				sse = sse+ self.sampleEr(Desired)
			 
			rmse = np.sqrt(sse/self.TrainData.shape[0]*self.Top[2])

			if rmse < bestRMSE:
				 bestRMSE = rmse
				 self.saveKnowledge() 
				 (bestRMSE,bestTrain) = self.TestNetwork(self.TrainData,   trainTolerance)
				 #Print(bestRMSE, bestTrain)

			Er = np.append(Er, rmse)
			
			epoch=epoch+1  

		return (Er,bestRMSE, bestTrain, epoch) 


In [ ]:
# gradient descent optimization with adam for a two-dimensional test function

'''这段代码实现了使用Adam优化算法来最小化一个简单的二维目标函数f(x, y) = x^2 + y^2。Adam是一种改进的梯度下降算法，它结合了动量（momentum）和自适应学习率的方法来加速优化。

以下是代码的主要步骤解释：

目标函数：定义了一个简单的目标函数objective(x, y)，其值是x和y的平方和。这是一个典型的二次函数，形状为抛物面，最小值位于原点(0, 0)。

导数函数：定义了目标函数的导数（梯度）derivative(x, y)，返回的是每个变量的偏导数。对于目标函数来说，偏导数分别是2x和2y。

Adam优化算法：

函数adam用于实现Adam优化算法。
首先随机生成一个初始点x，在给定范围内选取。
初始化动量m和平方梯度v，用于存储一阶和二阶动量。
迭代n_iter次，每次计算梯度g，然后更新动量m和平方梯度v。
使用修正后的动量mhat和vhat来更新每个变量的值x，使其逐步靠近函数的最小值。
每次迭代后输出当前点的函数值，以查看优化进度。
参数设置：

设置搜索范围bounds为[-1, 1]，即每个变量的初始值在-1到1之间。
n_iter为60，表示算法迭代60次。
alpha为步长（学习率），beta1和beta2分别为动量和均方梯度的指数加权平均系数。
输出：

在每次迭代中，代码会打印当前的迭代次数、当前点的位置以及目标函数值。
最后输出优化得到的点及其对应的函数值。
Adam算法通过结合动量和自适应学习率，能够在一定程度上加快梯度下降的收敛速度，并防止陷入局部极小值。这段代码演示了如何从随机初始点开始，通过多次迭代不断逼近目标函数的最小值。'''

from math import sqrt
from numpy import asarray
from numpy.random import rand
from numpy.random import seed
 
# objective function
def objective(x, y):
	return x**2.0 + y**2.0
 
# derivative of objective function
def derivative(x, y):
	return asarray([x * 2.0, y * 2.0])
 
# gradient descent algorithm with adam
def adam(objective, derivative, bounds, n_iter, alpha, beta1, beta2, eps=1e-8):
	# generate an initial point
	x = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
	score = objective(x[0], x[1])
	# initialize first and second moments
	m = [0.0 for _ in range(bounds.shape[0])]
	v = [0.0 for _ in range(bounds.shape[0])]
	# run the gradient descent updates
	for t in range(n_iter):
		# calculate gradient g(t)
		g = derivative(x[0], x[1])
		# build a solution one variable at a time
		for i in range(x.shape[0]):
			# m(t) = beta1 * m(t-1) + (1 - beta1) * g(t)
			m[i] = beta1 * m[i] + (1.0 - beta1) * g[i]
			# v(t) = beta2 * v(t-1) + (1 - beta2) * g(t)^2
			v[i] = beta2 * v[i] + (1.0 - beta2) * g[i]**2
			# mhat(t) = m(t) / (1 - beta1(t))
			mhat = m[i] / (1.0 - beta1**(t+1))
			# vhat(t) = v(t) / (1 - beta2(t))
			vhat = v[i] / (1.0 - beta2**(t+1))
			# x(t) = x(t-1) - alpha * mhat(t) / (sqrt(vhat(t)) + eps)
			x[i] = x[i] - alpha * mhat / (sqrt(vhat) + eps)
		# evaluate candidate point
		score = objective(x[0], x[1])
		# report progress
		print('>%d f(%s) = %.5f' % (t, x, score))
	return [x, score]
 
# seed the pseudo random number generator
seed(1)
#source: https://machinelearningmastery.com/adam-optimization-from-scratch/

# define range for input
bounds = asarray([[-1.0, 1.0], [-1.0, 1.0]])
# define the total iterations
n_iter = 60
# steps size
alpha = 0.02
# factor for average gradient
beta1 = 0.8
# factor for average squared gradient
beta2 = 0.999
# perform the gradient descent search with adam
best, score = adam(objective, derivative, bounds, n_iter, alpha, beta1, beta2)
print('Done!')
print('f(%s) = %f' % (best, score))

In [ ]:

# Rohitash Chandra, 2021 c.rohitash@gmail.conm
'''
这段代码实现了一个简单的深度神经网络类，包含多个全连接层。代码主要分为两个部分：Layers类和Network类，分别代表神经网络的各层以及整个神经网络的结构和训练过程。

下面是代码的详细解释：

1. Layers类
Layers类用于表示神经网络中的一个层。其目的是初始化每个神经网络层的权重、偏置以及一些用于存储输出和梯度的变量。

初始化方法 (__init__)：
参数first是输入神经元的数量，second是输出神经元的数量。
权重 (weights)：使用np.random.uniform(-0.5, 0.5, (first , second))随机初始化一个形状为(first, second)的权重矩阵，值在-0.5到0.5之间。
偏置 (bias)：偏置矩阵也被随机初始化为(-0.5, 0.5)的值，形状为(1, second)。
输出 (output)和梯度 (gradient)：用于存储这一层的输出和反向传播中的梯度，初始值为0。
2. Network类
Network类表示整个神经网络的结构和训练过程。

初始化方法 (__init__)：
topology：表示网络的拓扑结构，即每层的神经元数量，例如[4, 5, 3]表示输入层有4个神经元，隐藏层有5个神经元，输出层有3个神经元。
x_train、x_test、y_train、y_test：分别是训练集和测试集的特征和标签。
max_epocs：最大训练轮数（epochs）。
min_error：定义的最小误差，用于决定训练是否终止。
learn_rate：用于随机梯度下降（SGD）的学习率。
Adam学习率 (adam_learnrate)：初始化为0.05，敏感于Adam优化算法。
end_index：记录最后一层的索引，方便后续的操作。
层的初始化：使用self.layer = [None]*20来预先创建20个可能的层（假设神经网络最多有20层），然后通过for循环根据topology初始化实际的层。
第一个元素是一个特殊的层，用来存储输入特征。
其余的层根据拓扑结构初始化，每层的输入大小等于前一层的输出大小，输出大小等于当前层的神经元数量。
'''
#https://github.com/sydney-machine-learning/multilayerperceptron-sgd-adam/blob/main/fnn-multiplelayers.py

import matplotlib.pyplot as plt
import numpy as np 
import random
import time


from numpy import *


from sklearn.preprocessing import Normalizer

from sklearn.model_selection import train_test_split 


#Source: https://github.com/sydney-machine-learning/multilayerperceptron-sgd-adam/blob/main/fnn-multiplelayers.py




class Layers:

	def __init__(self, first, second,adam_learnrate=None):  
		#self.number = first

		self.weights = np.random.uniform(-0.5, 0.5, (first , second))   
		self.bias = np.random.uniform(-0.5,0.5, (1, second))  # bias second layer

		self.output = np.zeros(second) # output of  layer 
		self.gradient = np.zeros(second) # gradient of layer 
 

class Network:

	def __init__(self, topology, x_train, x_test, y_train, y_test, max_epocs,   min_error, learn_rate): # this is called construtor
		self.topology  = topology   

		self.output_activationfunc = 'sigmoid' 

		self.max_epocs = max_epocs # max epocs
		#self.TrainData = Train
		#self.TestData = Test

		self.x_train = x_train
		self.x_test = x_test
		self.y_train = y_train
		self.y_test = y_test


		self.num_samples =  x_train.shape[0] 

		self.sgdlearn_rate  = learn_rate
 

		self.min_error = min_error 
		 
		np.random.seed()   

		self.adam_learnrate = 0.05 #sensitive for adam
 

		self.end_index = len(self.topology)-1


		self.layer = [None]*20  # create list of Layers objects ( just max size of 20 for now - assuming a very deep neural network )

		print(self.topology,  ' topology')

		self.layer[0] = Layers(1,1, 0) # this is just for layer where input features are stored

		for n in range(1,  len(self.topology)):  
			self.layer[n] = Layers(self.topology[n-1],self.topology[n], self.adam_learnrate)

# Week5

In [ ]:
'''
这段代码定义了一个MCMC类，用于实现基于马尔可夫链蒙特卡罗（MCMC）方法的神经网络训练和评估。MCMC是一种统计方法，用于从复杂的概率分布中采样，通常用于贝叶斯推理和神经网络的优化。

以下是代码的详细解释：

MCMC类
1. 初始化方法 (__init__)
samples：采样的数量，用于MCMC过程。
topology：神经网络的拓扑结构，定义了输入、隐藏和输出层的神经元数量。
traindata 和 testdata：训练和测试数据，用于模型评估。
regression：如果为True，表示这是回归问题；如果为False，则表示是分类问题。
random.seed()：用来初始化随机数生成器，以确保每次运行的结果不同。
2. RMSE计算方法 (rmse)
函数定义：rmse(self, predictions, targets)。
计算预测值和真实目标值之间的均方根误差（RMSE），用于评估模型的性能。
公式：np.sqrt(((predictions - targets) ** 2).mean())，通过计算误差平方的均值并取平方根得到RMSE。
3. 似然函数 (likelihood_func)
函数定义：likelihood_func(self, model, data, w, tausq)。
该函数用于计算模型在给定权重w下的似然值。
y：目标值，从数据集中提取。
fx：通过模型的evaluate_proposal()函数使用当前权重w来计算模型的预测值。
RMSE计算：调用前面定义的rmse方法计算预测的精度。
损失计算：使用高斯似然函数，计算负对数似然损失。
loss = -0.5 * np.log(2 * math.pi * tausq) - 0.5 * np.square(y - fx) / tausq。
其中tausq表示噪声的方差。
返回值：返回损失的总和、模型的预测值和RMSE精度。
4. 先验似然函数 (prior_likelihood)
函数定义：prior_likelihood(self, sigma_squared, nu_1, nu_2, w, tausq)。
该函数计算模型参数w的先验似然值，用于贝叶斯推理中的先验分布。
参数：
sigma_squared：模型权重的方差，控制先验的分布宽度。
nu_1 和 nu_2：用于计算逆伽马分布（inverse gamma distribution）的参数。
w：神经网络的权重。
tausq：输出的噪声方差。
先验似然的计算：
param = self.topology[0] + 1：计算模型参数的数量。
部分1（多元正态分布部分）：计算多元正态分布的对数先验。
part1 = -1 * (param / 2) * np.log(sigma_squared)：根据方差的对数，计算常数项。
part2 = 1 / (2 * sigma_squared) * (sum(np.square(w)))：计算权重平方和的部分。
multivariate_normal = part1 - part2：计算多元正态分布的对数先验。
部分2（逆伽马分布部分）：
inverse_gamma = -(1 + nu_1) * np.log(tausq) - (nu_2 / tausq)：计算逆伽马分布的对数先验。
对数损失 (log_loss)：
log_loss = multivariate_normal + inverse_gamma：将两个对数似然部分相加，得到总的先验似然值。
返回值：返回计算得到的先验对数损失。
'''
class MCMC:
	def __init__(self, samples, traindata, testdata, topology, regression):
		self.samples = samples  # NN topology [input, hidden, output]
		self.topology = topology  # max epocs
		self.traindata = traindata  #
		self.testdata = testdata
		random.seed() 
		self.regression = regression # False means classification

	def rmse(self, predictions, targets):
		return np.sqrt(((predictions - targets) ** 2).mean())
 
	def likelihood_func(self, model, data, w, tausq):
		y = data[:, self.topology[0]]
		fx = model.evaluate_proposal(data, w) 
		accuracy = self.rmse(fx, y) #RMSE 
		loss = -0.5 * np.log(2 * math.pi * tausq) - 0.5 * np.square(y - fx) / tausq 
		return [np.sum(loss), fx, accuracy]

	def prior_likelihood(self, sigma_squared, nu_1, nu_2, w, tausq): 
		param = self.topology[0]  + 1 # number of parameters in model
		part1 = -1 * (param / 2) * np.log(sigma_squared)
		part2 = 1 / (2 * sigma_squared) * (sum(np.square(w)))
        multivariate_normal = part1 - part2
        inverse_gamma = -(1 + nu_1) * np.log(tausq) - (nu_2 / tausq)
		log_loss = multivariate_normal + inverge_gamma
		return log_loss

In [ ]:
 # by R. Chandra
 #https://github.com/rohitash-chandra/Bayesian_logisticregression

'''
这段代码实现了一个简单的线性模型，用于回归或分类任务（如逻辑回归）。代码通过定义lin_model类，创建了一个模型，包含训练数据和测试数据的处理，以及前向传播和预测的实现。

lin_model类解释
lin_model类的作用是实现一个简单的线性模型，该模型可以用于回归或分类任务，具体取决于是否使用激活函数。

1. 初始化方法 (__init__)
num_epocs：训练的最大迭代次数（epochs）。
train_data 和 test_data：分别是训练数据和测试数据。
num_features：输入数据的特征数量。
learn_rate：学习率，用于梯度更新。
activation：是否使用激活函数。如果为False，则使用Sigmoid激活函数。
初始化权重 (self.w) 和偏置 (self.b)：
权重w初始化为在-0.5到0.5之间随机生成的值，大小为num_features。
偏置b也是在-0.5到0.5之间随机生成的值，大小为输出的数量（即num_outputs）。
数据的形状：
self.num_outputs为输出类别的数量，等于训练数据列数减去特征的数量。
self.num_train是训练数据的样本数量。
2. 激活函数 (activation_func)
函数定义：activation_func(self, z_vec)。
如果self.use_activation为False，则使用Sigmoid函数：y = 1 / (1 + np.exp(z_vec))。
如果self.use_activation为True，则不进行激活，直接输出输入值z_vec。
返回激活后的输出值。
3. 预测函数 (predict)
函数定义：predict(self, x_vec)。
计算过程：
使用线性函数z_vec = x_vec.dot(self.w) - self.b计算输出。
将结果传递给激活函数：output = self.activation_func(z_vec)。
返回最终的输出output。
4. 均方误差函数 (squared_error)
函数定义：squared_error(self, prediction, actual)。
计算过程：
计算预测值与真实值之间的均方误差（MSE）：np.sum(np.square(prediction - actual))/prediction.shape[0]。
返回误差值。均方误差用于衡量预测值与真实值之间的差异。
5. 编码函数 (encode)
函数定义：encode(self, w)。
作用：将输入的权重w解码为模型的权重和偏置。
self.w = w[0:self.num_features]：将输入的前num_features个值赋给模型的权重。
self.b = w[self.num_features]：将权重数组的最后一个值赋给偏置b。
6. 提案评估函数 (evaluate_proposal)
函数定义：evaluate_proposal(self, data, w)。
作用：通过给定的权重w对输入数据进行评估，并返回预测值。
计算过程：
首先使用encode(w)方法将输入权重解码为模型的权重和偏置。
创建fx数组，用于存储每个样本的预测值。
对每个样本进行预测：
input_instance提取输入特征。
使用predict(input_instance)计算模型对样本的预测值。
将预测值存储在fx中。
返回所有样本的预测值数组fx。
代码的整体流程
导入模块：导入了一些常用的库，包括numpy、math和matplotlib，用于数值计算、数学运算和绘图。
lin_model类的定义：
lin_model类用于创建一个线性模型，并初始化相应的参数。
包含了前向传播的实现（predict方法）、激活函数（activation_func方法）、计算误差（squared_error方法）等。
提供了权重的编码和评估提案的方法（encode和evaluate_proposal）。
激活函数的选择：
可以选择是否使用激活函数，如果使用，则采用Sigmoid激活函数，否则输出线性值。
'''


import numpy as np
import random
import math
import matplotlib.pyplot as plt
from math import exp

class lin_model:

	def __init__(self, num_epocs, train_data, test_data, num_features, learn_rate, activation):
		self.train_data = train_data
		self.test_data = test_data 
		self.num_features = num_features
		self.num_outputs = self.train_data.shape[1] - num_features 
		self.num_train = self.train_data.shape[0]
		self.w = np.random.uniform(-0.5, 0.5, num_features)  # in case one output class 
		self.b = np.random.uniform(-0.5, 0.5, self.num_outputs) 
		self.learn_rate = learn_rate
		self.max_epoch = num_epocs
		self.use_activation = activation #SIGMOID # 1 is  sigmoid , 2 is step, 3 is linear 
		self.out_delta = np.zeros(self.num_outputs)
		self.activation = activation
 
	def activation_func(self,z_vec):
		if self.use_activation == False:
			y =  1 / (1 + np.exp(z_vec)) # sigmoid/logistic
		else:
			y = z_vec 
		return y

	def predict(self, x_vec ): 
		z_vec = x_vec.dot(self.w) - self.b 
		output = self.activation_func(z_vec) # Output  
		return output
	


	def squared_error(self, prediction, actual):
		return  np.sum(np.square(prediction - actual))/prediction.shape[0]# to cater more in one output/class
 
	def encode(self, w): # get  the parameters and encode into the model
		 
		self.w =  w[0:self.num_features]
		self.b = w[self.num_features] 

	def evaluate_proposal(self, data, w):  # BP with SGD (Stocastic BP)

		self.encode(w)  # method to encode w and b
		fx = np.zeros(data.shape[0]) 

		for s in range(0, data.shape[0]):  
				i = s #random.randint(0, data.shape[0]-1)  (we dont shuffle in this implementation)
				input_instance  =  data[i,0:self.num_features]  
				actual  = data[i,self.num_features:]  
				prediction = self.predict(input_instance)  
				fx[s] = prediction

		return fx

In [ ]:
'''
1. 初始化MCMC采样
样本大小 (testsize, trainsize, samples)：

testsize：测试集样本数量。
trainsize：训练集样本数量。
samples：采样次数，即需要从后验分布中采样多少组参数。
数据划分 (x_test, x_train)：

x_test 和 x_train 使用np.linspace(0, 1, num=...)生成线性分布的输入样本。
提取标签 (y_test, y_train)：

y_test 和 y_train 分别为测试和训练数据的真实标签。
使用self.topology[0]提取出输出值。
权重大小 (w_size)：

权重大小w_size等于输入和输出神经元数量之和。对于一个简单的线性模型，权重数等于输入特征数，偏置也作为一个单独的参数。
初始化采样存储：

pos_w：用于存储所有采样的权重和偏置，形状为(samples, w_size)。
pos_tau：用于存储噪声精度（逆方差），形状为(samples, 1)。
fxtrain_samples 和 fxtest_samples：分别用于存储训练集和测试集上的预测结果，维度为(samples, trainsize)和(samples, testsize)。
rmse_train 和 rmse_test：用于存储每次采样后计算的训练集和测试集的RMSE。
2. 初始化权重和步长
权重初始化 (w, w_proposal)：

w和w_proposal是用于当前采样和提案的权重，使用标准正态分布初始化。
步长设置 (step_w, step_eta)：

step_w：控制权重的变化范围。
step_eta：控制噪声精度参数的变化范围。
3. 线性模型的初始化
使用lin_model类初始化一个模型实例model。
传入的参数包括训练数据、测试数据、特征数量和一个固定的学习率。
注意：max_epoch设置为0，表示在sampler中并不涉及显式的训练过程，而是通过MCMC采样来优化权重。
4. 计算初始预测和似然
预测值计算 (pred_train, pred_test)：

使用模型的evaluate_proposal()方法，传入当前权重w，计算训练集和测试集的预测值。
计算噪声精度 (eta, tau_pro)：

eta：计算训练集预测误差的方差的对数，以此得到噪声精度的一个度量。
tau_pro：使用指数函数计算精度参数，即精度为方差的逆数。
5. 计算初始先验似然和数据似然
先验似然 (prior_likelihood)：

使用self.prior_likelihood()方法计算当前权重的先验似然。
参数包括sigma_squared、nu_1、nu_2等，用于表示先验分布的超参数。
数据似然 (likelihood)：

使用self.likelihood_func()方法计算训练集的似然值。
返回似然值、训练集的预测值和RMSE。
6. 打印初始状态并准备MCMC循环
打印初始信息：打印初始权重的评估信息以及初始似然值，以便查看初始模型的性能。
变量naccept：用于记录MCMC采样过程中被接受的提案数，初始化为0。
代码的整体流程
初始化采样参数和存储矩阵：
包括存储权重和偏置、预测值以及每个采样的均方误差。
初始化权重和噪声精度：
初始化权重和用于提案的权重，以及步长参数。
创建线性模型实例：
使用训练数据和测试数据初始化模型。
计算初始似然和先验：
通过模型预测和计算方差，得到初始的似然和先验。
'''
def sampler(self):

		# ------------------- initialize MCMC
		testsize = self.testdata.shape[0]
		trainsize = self.traindata.shape[0]
		samples = self.samples

		x_test = np.linspace(0, 1, num=testsize)
		x_train = np.linspace(0, 1, num=trainsize)

		#self.topology  # [input,   output]
		y_test = self.testdata[:, self.topology[0]]
		y_train = self.traindata[:, self.topology[0]]
	  
		w_size = self.topology[0]  + self.topology[1]  # num of weights and bias (eg. 4 + 1 in case of time series problems used)

		pos_w = np.ones((samples, w_size))  # posterior of all weights and bias over all samples
		pos_tau = np.ones((samples, 1))

		fxtrain_samples = np.ones((samples, trainsize))  # fx of train data over all samples
		fxtest_samples = np.ones((samples, testsize))  # fx of test data over all samples
		rmse_train = np.zeros(samples)
		rmse_test = np.zeros(samples)

		w = np.random.randn(w_size)
		w_proposal = np.random.randn(w_size)

		step_w = 0.02;  # defines how much variation you need in changes to w
		step_eta = 0.01;  
		# eta is an additional parameter to cater for noise in predictions (Gaussian likelihood). 
		# note eta is used as tau in the sampler to consider log scale. 
		# eta is not used in multinomial likelihood. 
 

		model = lin_model(0 ,  self.traindata, self.testdata, self.topology[0], 0.1, self.regression) 

		pred_train = model.evaluate_proposal(self.traindata, w)
		pred_test = model.evaluate_proposal(self.testdata, w)

		eta = np.log(np.var(pred_train - y_train))
		tau_pro = np.exp(eta)

		print('evaluate Initial w')

		sigma_squared = 5  # considered by looking at distribution of  similar trained  models - i.e distribution of weights and bias
		nu_1 = 0
		nu_2 = 0

		prior_likelihood = self.prior_likelihood(sigma_squared, nu_1, nu_2, w, tau_pro)  # takes care of the gradients

		[likelihood, pred_train, rmsetrain] = self.likelihood_func(model, self.traindata, w, tau_pro)

		print(likelihood, ' initial likelihood')
		[likelihood_ignore, pred_test, rmsetest] = self.likelihood_func(model, self.testdata, w, tau_pro)


		naccept = 0  

In [ ]:
'''
这段代码实现了一种基于马尔可夫链蒙特卡罗（MCMC）方法的采样算法，用于从后验分布中进行采样，目的是找到模型参数的最佳估计。这里使用的是一种Metropolis-Hastings（MH）采样算法。以下是代码的逐步解释：

循环采样过程：

使用 for i in range(samples - 1) 进行多次迭代来生成样本。
在每次迭代中生成新的参数 w_proposal 和 eta_pro，这些新参数是通过在当前参数值的基础上加上随机噪声得到的。
计算新的似然和先验：

通过 self.likelihood_func 计算新的候选参数下的似然值 likelihood_proposal 以及训练和测试数据上的预测结果。
计算候选参数的先验值 prior_prop，通过调用 self.prior_likelihood。
Metropolis-Hastings准则：

计算似然差值 diff_likelihood 和先验差值 diff_priorliklihood。
使用这些差值来计算接受概率 mh_prob，并用一个随机数 u 来决定是否接受候选参数。
如果 u < mh_prob，则接受候选参数并更新参数的值，否则保持现有参数。
保存采样结果：

将当前迭代得到的参数、预测值、均方误差（RMSE）等保存在相应的数组中，以便在后续分析中使用。
计算接受率：

计算采样的接受率 accept_ratio，即有多少次候选参数被接受。
后处理：

扔掉“燃烧期”（burn-in period）的样本，用于去除初始不稳定的采样。
最后，计算并打印训练和测试集上的RMSE均值和标准差，来衡量模型的预测性能。
总结来说，这段代码实现了一个基于Metropolis-Hastings算法的参数采样过程，用于优化某个模型的参数，并通过均方误差（RMSE）来评估模型的性能。主要步骤包括提议新参数、计算接受概率、根据接受概率更新参数，以及记录采样过程的结果。
'''


		for i in range(samples - 1):

			w_proposal = w + np.random.normal(0, step_w, w_size)

			eta_pro = eta + np.random.normal(0, step_eta, 1)
			tau_pro = math.exp(eta_pro)

			[likelihood_proposal, pred_train, rmsetrain] = self.likelihood_func(model, self.traindata, w_proposal, tau_pro)
			[likelihood_ignore, pred_test, rmsetest] = self.likelihood_func(model, self.testdata, w_proposal, tau_pro)

			# likelihood_ignore  refers to parameter that will not be used in the alg.

			prior_prop = self.prior_likelihood(sigma_squared, nu_1, nu_2, w_proposal, tau_pro)  # takes care of the gradients

			diff_likelihood = likelihood_proposal - likelihood # since we using log scale: based on https://www.rapidtables.com/math/algebra/Logarithm.html
			diff_priorliklihood = prior_prop - prior_likelihood

			mh_prob = min(1, math.exp(diff_likelihood + diff_priorliklihood))

			u = random.uniform(0, 1)

			if u < mh_prob:
				# Update position
				#print    (i, ' is accepted sample')
				naccept += 1
				likelihood = likelihood_proposal
				prior_likelihood = prior_prop
				w = w_proposal
				eta = eta_pro
				rmse_train[i + 1,] = rmsetrain
				rmse_test[i + 1,] = rmsetest


				#print (likelihood, prior_likelihood, rmsetrain, rmsetest, w, 'accepted')

				pos_w[i + 1,] = w_proposal
				pos_tau[i + 1,] = tau_pro
				fxtrain_samples[i + 1,] = pred_train
				fxtest_samples[i + 1,] = pred_test 

			else:
				pos_w[i + 1,] = pos_w[i,]
				pos_tau[i + 1,] = pos_tau[i,]
				fxtrain_samples[i + 1,] = fxtrain_samples[i,]
				fxtest_samples[i + 1,] = fxtest_samples[i,] 
				rmse_train[i + 1,] = rmse_train[i,]
				rmse_test[i + 1,] = rmse_test[i,]

		 
		accept_ratio = naccept / (samples * 1.0) * 100


		print(accept_ratio, '% was accepted')

		burnin = 0.25 * samples  # use post burn in samples

		pos_w = pos_w[int(burnin):, ]
		pos_tau = pos_tau[int(burnin):, ] 
		rmse_train = rmse_train[int(burnin):]
		rmse_test = rmse_test[int(burnin):] 


		rmse_tr = np.mean(rmse_train)
		rmsetr_std = np.std(rmse_train)
		rmse_tes = np.mean(rmse_test)
		rmsetest_std = np.std(rmse_test)
		print(rmse_tr, rmsetr_std, rmse_tes, rmsetest_std, ' rmse_tr, rmsetr_std, rmse_tes, rmsetest_std')

In [ ]:
'''
这段代码包含了两个函数，likelihood_func 和 prior_likelihood，用于计算贝叶斯神经网络中采样参数的似然和先验，帮助实现参数的贝叶斯优化。

1. likelihood_func
该函数计算神经网络在给定数据和权重下的似然值。具体步骤如下：

参数：

neuralnet：神经网络模型。
data：输入数据。
w：权重向量。
tausq：噪声精度（逆方差）。
实现：

提取目标值：y = data[:, self.topology[0]]，从数据中提取目标值（真实输出）。
计算模型预测：fx = neuralnet.evaluate_proposal(data, w)，使用给定的权重 w 来计算模型的预测值。
计算均方根误差（RMSE）：rmse = self.rmse(fx, y)，计算预测值和目标值之间的RMSE。
计算似然：loss = -0.5 * np.log(2 * math.pi * tausq) - 0.5 * np.square(y - fx) / tausq，这一步计算对数似然，假设误差服从高斯分布。
返回值：return [np.sum(loss), fx, rmse]，返回对数似然的和、模型预测值和RMSE。
2. prior_likelihood
该函数用于计算给定权重和噪声精度下的先验对数概率，用于评估参数在先验分布下的合理性。
参数：

sigma_squared：权重的先验方差。
nu_1 和 nu_2：控制精度的先验参数。
w：权重向量。
tausq：噪声精度。
实现：

提取神经网络结构信息：
h = self.topology[1]：隐藏神经元的数量。
d = self.topology[0]：输入神经元的数量。
计算先验的第一部分：part1 = -1 * ((d * h + h + 2) / 2) * np.log(sigma_squared)，先验概率的第一部分涉及对数方差，反映了权重的复杂度。
计算先验的第二部分：part2 = 1 / (2 * sigma_squared) * (sum(np.square(w)))，这是权重的平方和项，代表L2正则化。
计算总的对数先验：log_loss = part1 - part2 - (1 + nu_1) * np.log(tausq) - (nu_2 / tausq)，包含了权重的对数先验和精度参数的对数先验。
返回值：return log_loss，返回先验对数概率。
总结
likelihood_func 用于计算在给定数据和权重下模型的对数似然，反映了模型在给定权重下对数据的拟合程度。
prior_likelihood 用于计算权重和噪声精度的先验对数概率，反映了参数符合先验分布的程度。
'''
# https://github.com/sydney-machine-learning/Bayesianneuralnetworks-MCMC-tutorial/blob/main/code/BNN_reg_cls.py   
    
    def likelihood_func(self, neuralnet, data, w, tausq):
		y = data[:, self.topology[0]]
		fx = neuralnet.evaluate_proposal(data, w)
		rmse = self.rmse(fx, y)
		loss = -0.5 * np.log(2 * math.pi * tausq) - 0.5 * np.square(y - fx) / tausq
		return [np.sum(loss), fx, rmse]

	def prior_likelihood(self, sigma_squared, nu_1, nu_2, w, tausq):
		h = self.topology[1]  # number hidden neurons
		d = self.topology[0]  # number input neurons
		part1 = -1 * ((d * h + h + 2) / 2) * np.log(sigma_squared)
		part2 = 1 / (2 * sigma_squared) * (sum(np.square(w)))
		log_loss = part1 - part2  - (1 + nu_1) * np.log(tausq) - (nu_2 / tausq)
		return log_loss

In [ ]:
'''
1. decode(self, w)
这个函数用于将给定的一维权重向量 w 解码成神经网络的权重矩阵和偏置向量。

参数：

w：包含所有权重和偏置的一维向量。
实现：

计算隐藏层的权重大小 w_layer1size 和输出层的权重大小 w_layer2size。
第一层权重：w_layer1 是第一个隐藏层的权重部分，通过 np.reshape 将其重塑为 W1 矩阵。
第二层权重：w_layer2 是输出层的权重部分，通过 np.reshape 将其重塑为 W2 矩阵。
偏置向量：
self.B1 是隐藏层的偏置。
self.B2 是输出层的偏置。
这个函数用于解码整个权重向量，使神经网络可以进行前向传播和后向传播。

2. langevin_gradient(self, data, w, depth)
这个函数实现了使用随机梯度下降（SGD）进行后向传播，以计算Langevin梯度，用于更新权重。

参数：

data：输入数据集。
w：权重向量。
depth：训练的深度，即多少次迭代。
实现：

使用 decode(w) 解码给定的权重向量。
初始化输入和输出：Input 和 Desired 用于存储每个样本的输入和期望输出。
进行多次迭代（由 depth 决定），每次迭代中：
遍历数据集，使用 ForwardPass 进行前向传播，计算输出。
使用 BackwardPass 进行后向传播，以更新权重。
调用 self.encode() 来重新编码更新后的权重为一维向量。
返回更新后的权重 w_updated。
这个函数的目的是通过Langevin梯度来更新模型参数，从而提高模型性能。

3. evaluate_proposal(self, data, w)
该函数用于给定数据和权重，评估模型的预测结果。

参数：

data：输入数据。
w：权重向量。
实现：

使用 decode(w) 来解码给定的权重向量。
初始化输入和输出：Input 用于存储输入样本，fx 用于存储模型的预测输出。
进行前向传播：
遍历数据集中的所有样本。
使用 ForwardPass 对每个样本进行前向传播，得到输出 self.out。
将输出存储在 fx 中。
返回预测结果 fx。
这个函数的主要目的是使用给定权重对数据集进行预测，以便评估模型在当前权重下的表现。

总结
decode：将一维的权重向量解码为神经网络各层的权重矩阵和偏置向量，使其可以用于计算。
langevin_gradient：使用SGD进行后向传播，通过Langevin梯度来更新权重，提高模型性能。
evaluate_proposal：给定数据和权重，使用神经网络对输入数据进行前向传播，并返回预测结果。
'''
# https://github.com/sydney-machine-learning/Bayesianneuralnetworks-MCMC-tutorial/blob/main/code/BNN_reg_cls.py   
    def decode(self, w):
		w_layer1size = self.Top[0] * self.Top[1]
		w_layer2size = self.Top[1] * self.Top[2]

		w_layer1 = w[0:w_layer1size]
		self.W1 = np.reshape(w_layer1, (self.Top[0], self.Top[1]))

		w_layer2 = w[w_layer1size:w_layer1size + w_layer2size]
		self.W2 = np.reshape(w_layer2, (self.Top[1], self.Top[2]))
		self.B1 = w[w_layer1size + w_layer2size:w_layer1size + w_layer2size + self.Top[1]]
		self.B2 = w[w_layer1size + w_layer2size + self.Top[1]:w_layer1size + w_layer2size + self.Top[1] + self.Top[2]]

	def langevin_gradient(self, data, w, depth):  # BP with SGD (Stocastic BP)

		self.decode(w)  # method to decode w into W1, W2, B1, B2.
		size = data.shape[0]

		Input = np.zeros((1, self.Top[0]))  # temp hold input
		Desired = np.zeros((1, self.Top[2]))
		fx = np.zeros(size)

		for i in xrange(0, depth):
			for i in xrange(0, size):
				pat = i
				Input = data[pat, 0:self.Top[0]]
				Desired = data[pat, self.Top[0]:]
				self.ForwardPass(Input)
				self.BackwardPass(Input, Desired)

		w_updated = self.encode()

		return  w_updated

	def evaluate_proposal(self, data, w ):  # BP with SGD (Stocastic BP)

		self.decode(w)  # method to decode w into W1, W2, B1, B2.
		size = data.shape[0]

		Input = np.zeros((1, self.Top[0]))  # temp hold input
		Desired = np.zeros((1, self.Top[2]))
		fx = np.zeros(size)

		for i in xrange(0, size):  # to see what fx is produced by your current weight update
			Input = data[i, 0:self.Top[0]]
			self.ForwardPass(Input)
			fx[i] = self.out

		return fx


In [ ]:
'''
这段代码实现了基于Metropolis-Hastings (MH) 算法的参数采样过程，其中结合了Langevin梯度，用于改进采样效率。这种方法用于贝叶斯神经网络中的马尔可夫链蒙特卡罗 (MCMC) 采样，以探索参数空间。以下是代码的详细解释：

1. Langevin 梯度的使用
在这个采样过程中，有时会使用Langevin梯度来改进采样效率。Langevin梯度结合了梯度信息以更好地指导参数更新，而不只是纯粹的随机步进。

lx = np.random.uniform(0,1,1)：

生成一个在0到1之间的随机数，用于决定当前迭代是否使用Langevin梯度。
if (self.use_langevin_gradients is True) and (lx < self.l_prob)：

如果设置了使用Langevin梯度（self.use_langevin_gradients为True），并且生成的随机数 lx 小于 self.l_prob，则进行Langevin梯度步骤。
w_gd = neuralnet.langevin_gradient(self.traindata, w.copy(), self.sgd_depth)：

使用Langevin梯度 (langevin_gradient) 更新权重 w，得到梯度更新后的权重 w_gd。
w_proposal = np.random.normal(w_gd, step_w, w_size)：

使用更新后的权重 w_gd 为均值，进行一个随机扰动，生成候选权重 w_proposal。
计算反向提议概率差值：

w_prop_gd = neuralnet.langevin_gradient(self.traindata, w_proposal.copy(), self.sgd_depth)：
使用 w_proposal 再次计算Langevin梯度。
wc_delta = (w - w_prop_gd) 和 wp_delta = (w_proposal - w_gd)：
计算从候选权重和当前权重之间的差异向量。
first 和 second：
first 代表从 w_prop_gd 到 w 的提议概率的负对数值，second 代表从 w_gd 到 w_proposal 的提议概率的负对数值。
使用这些差值来计算两个提议方向之间的概率差异（diff_prop），这在Langevin方法中是必要的，以确保MH步骤中的对称性。
diff_prop = first - second：

计算提议概率的对数差异。
2. 不使用Langevin梯度时的采样
如果当前不使用Langevin梯度，则 diff_prop = 0，并通过随机扰动直接生成候选权重 w_proposal。
3. 更新噪声参数 eta
eta_pro = eta + np.random.normal(0, step_eta, 1)：
更新噪声参数 eta，通过在当前值基础上加一个随机扰动生成新的 eta_pro。
tau_pro = math.exp(eta_pro)：
计算噪声精度 tau_pro，是 eta_pro 的指数形式。
4. 计算似然、先验和MH概率
似然计算：

[likelihood_proposal, pred_train, rmsetrain] = self.likelihood_func(...)：计算候选参数下的似然值。
[likelihood_ignore, pred_test, rmsetest] = self.likelihood_func(...)：计算候选参数下在测试集上的似然值（忽略对MH步骤的影响）。
先验计算：

prior_prop = self.prior_likelihood(...)：计算候选参数下的先验对数概率。
计算差值：

diff_prior = prior_prop - prior_current：计算先验的差值。
diff_likelihood = likelihood_proposal - likelihood：计算似然的差值。
5. 计算Metropolis-Hastings (MH) 概率
mh_prob = min(1, math.exp(diff_likelihood + diff_prior + diff_prop))：

计算MH概率，结合似然、先验和提议概率的差值。
异常处理：

由于 diff_likelihood + diff_prior + diff_prop 有可能非常大，导致 math.exp() 产生溢出错误，这里使用 try...except 来捕获 OverflowError，并在发生溢出时将 mh_prob 设为1，保证采样过程继续。
'''
#Source: https://github.com/sydney-machine-learning/Bayesianneuralnetworks-MCMC-tutorial/blob/main/code/BNN_reg_cls.py
for i in range(samples - 1):

    lx = np.random.uniform(0,1,1)

    if (self.use_langevin_gradients is True) and (lx< self.l_prob):  
        w_gd = neuralnet.langevin_gradient(self.traindata, w.copy(), self.sgd_depth) # Eq 8
        w_proposal = np.random.normal(w_gd, step_w, w_size) # Eq 7
        w_prop_gd = neuralnet.langevin_gradient(self.traindata, w_proposal.copy(), self.sgd_depth) 
        #first = np.log(multivariate_normal.pdf(w , w_prop_gd , sigma_diagmat)) #how likely is that you end up with w given w_prop_gd
        #second = np.log(multivariate_normal.pdf(w_proposal , w_gd , sigma_diagmat)) # this gives numerical instability - hence we give a simple implementation next that takes out log 

        wc_delta = (w- w_prop_gd) 
        wp_delta = (w_proposal - w_gd )
        sigma_sq = step_w

        first = -0.5 * np.sum(wc_delta  *  wc_delta  ) / sigma_sq  # this is wc_delta.T  *  wc_delta /sigma_sq
        second = -0.5 * np.sum(wp_delta * wp_delta ) / sigma_sq

        #in random-walk idealy first - second would be 0 or close to 0.
        diff_prop =  first - second  
        langevin_count = langevin_count + 1
    else:
        diff_prop = 0
        w_proposal = np.random.normal(w, step_w, w_size)

    eta_pro = eta + np.random.normal(0, step_eta, 1)
    tau_pro = math.exp(eta_pro)

    [likelihood_proposal, pred_train, rmsetrain] = self.likelihood_func(neuralnet, self.traindata, w_proposal,																tau_pro)
    [likelihood_ignore, pred_test, rmsetest] = self.likelihood_func(neuralnet, self.testdata, w_proposal,
                                                                    tau_pro) 

    prior_prop = self.prior_likelihood(sigma_squared, nu_1, nu_2, w_proposal,
                                        tau_pro)  # takes care of the gradients


    diff_prior = prior_prop - prior_current

    diff_likelihood = likelihood_proposal - likelihood

    #mh_prob = min(1, math.exp(diff_likelihood + diff_prior + diff_prop))

    try:
        mh_prob = min(1, math.exp(diff_likelihood+diff_prior+ diff_prop))

    except OverflowError as e:
        mh_prob = 1



In [ ]:
 '''
 这段代码实现了基于马尔可夫链蒙特卡洛(MCMC)方法中的Metropolis-Hastings采样，用于优化神经网络的参数。

代码首先检查是否要使用Langevin梯度（self.use_langevin_gradients），如果满足条件，则使用Langevin梯度更新参数。

w_gd = neuralnet.langevin_gradient(self.traindata, w.copy(), self.sgd_depth)：调用神经网络的langevin_gradient方法计算Langevin梯度。
w_proposal = np.random.normal(w_gd, step_w, w_size)：生成一个新的参数w_proposal，基于Langevin梯度（类似于随机游走，但带有梯度方向）。
代码计算Langevin梯度的对称性校正项diff_prop。

w_prop_gd表示w_proposal经过Langevin梯度后的值。
wc_delta和wp_delta分别是当前参数w与w_prop_gd的差异以及提议参数w_proposal与w_gd的差异。
使用这些差异计算概率对称性校正项first和second，并最终得到diff_prop。
代码最后一步是计算Metropolis-Hastings的接受概率mh_prob。

diff_prior表示提议分布与当前分布的先验差异。
diff_likelihood表示提议分布与当前分布的似然差异。
mh_prob是一个在0和1之间的概率，用于决定是否接受新的参数。这个概率由diff_likelihood、diff_prior以及对称性校正项diff_prop一起计算出来。
 '''
 if (self.use_langevin_gradients is True) and (lx< self.l_prob):  
        w_gd = neuralnet.langevin_gradient(self.traindata, w.copy(), self.sgd_depth) # Eq 8
        w_proposal = np.random.normal(w_gd, step_w, w_size) # Eq 7
        w_prop_gd = neuralnet.langevin_gradient(self.traindata, w_proposal.copy(), self.sgd_depth) 
        #first = np.log(multivariate_normal.pdf(w , w_prop_gd , sigma_diagmat))
        #(how likely is that you end up with w given w_prop_gd)
        #second = np.log(multivariate_normal.pdf(w_proposal , w_gd , sigma_diagmat)) 
        #(this gives numerical instability - hence we give a simple implementation next that takes out log)

        wc_delta = (w- w_prop_gd) 
        wp_delta = (w_proposal - w_gd )
        sigma_sq = step_w

        first = -0.5 * np.sum(wc_delta  *  wc_delta  ) / sigma_sq  
        second = -0.5 * np.sum(wp_delta * wp_delta ) / sigma_sq

        #in random-walk idealy first - second would be 0 or close to 0 
        #and hence we assume it cancels out
        diff_prop =  first - second  


# get likelihood and prior likelihood values (prior_prop and diff_likelihood)

    diff_prior = prior_prop - prior_current
    diff_likelihood = likelihood_proposal - likelihood
    mh_prob = min(1, math.exp(diff_likelihood + diff_prior + diff_prop))


In [ ]:
'''
1. loglikelihood(self, neuralnet, data, w, tausq)
这个函数计算神经网络的对数似然值，具体取决于问题类型是回归（regression）还是分类（classification）。

对于回归问题，调用gaussian_loglikelihood函数。
对于分类问题，调用multinomial_loglikelihood函数。
返回值是包含对数似然值、预测值以及性能指标（例如均方根误差或准确率）。
2. prior(self, sigma_squared, nu_1, nu_2, w, tausq)
这个函数用于计算先验分布的对数值。

如果是回归问题，调用prior_regression计算先验。
如果是分类问题，调用prior_classification计算先验。
返回先验的对数值。
3. gaussian_loglikelihood(self, neuralnet, data, w, tausq)
这个函数计算回归问题的对数似然值，假设误差服从高斯分布。

y是数据中的目标值。
fx, prob = neuralnet.evaluate_proposal(data, w)：计算神经网络在当前参数w下的输出预测fx，prob被忽略。
rmse = self.rmse(fx, y)：计算预测fx和目标值y之间的均方根误差。
对数似然值log_lhood是根据高斯分布公式计算的，使用y和fx的误差以及噪声参数tausq。
返回对数似然值、预测值和均方根误差。
4. prior_regression(self, sigma_squared, nu_1, nu_2, w, tausq)
这个函数计算回归问题中参数的先验对数概率。

先验分布假设权重和偏置是正态分布。
h和d分别表示隐藏层神经元和输入层神经元的数量。
part1和part2用于计算基于正态分布的先验对数概率，tausq部分也包含在先验中。
返回先验的对数损失log_loss。
5. multinomial_loglikelihood(self, neuralnet, data, w)
这个函数计算分类问题的对数似然值。

y是数据中的目标类别。
fx, prob = neuralnet.evaluate_proposal(data,w)：计算神经网络在当前参数w下的输出预测fx和对应的概率prob。
acc = self.accuracy(fx, y)：计算预测的准确率。
使用多项分布的对数似然公式，计算每个样本的对数似然值lhood，其中z[i, j]是用于标记类别的指示符。
返回对数似然值、预测值和准确率。
6. prior_classification(self, sigma_squared, nu_1, nu_2, w)
这个函数计算分类问题中参数的先验对数概率。

类似于prior_regression，但计算方式中会根据分类输出的结构有所不同。
先验同样假设权重服从正态分布。
log_loss是先验对数损失。
总结
loglikelihood和prior函数分别计算了不同类型问题的对数似然值和先验对数概率。
gaussian_loglikelihood和multinomial_loglikelihood分别用于回归和分类问题的似然值计算。
prior_regression和prior_classification用于回归和分类问题中先验对数概率的计算。
'''
#taken from: https://github.com/sydney-machine-learning/Bayesianneuralnetworks-MCMC-tutorial/blob/main/code/BNN_reg_cls.py

	def loglikelihood(self, neuralnet, data, w, tausq): 

		if self.prob == 'regression':
			[log_lhood, prediction, perf] = self.gaussian_loglikelihood(neuralnet, data, w, tausq)
		elif self.prob == 'classification':
			[log_lhood, prediction, perf] = self.multinomial_loglikelihood(neuralnet, data, w)

		return [log_lhood, prediction, perf]

	def prior(self, sigma_squared, nu_1, nu_2, w, tausq): 

		if self.prob == 'regression':
			logprior = self.prior_regression(sigma_squared, nu_1, nu_2, w, tausq)
		elif self.prob == 'classification':
			logprior = self.prior_classification(sigma_squared, nu_1, nu_2, w)

		return logprior

	def gaussian_loglikelihood(self, neuralnet, data, w, tausq): 

		y = data[:, self.topology[0]]
		fx, prob = neuralnet.evaluate_proposal(data, w) #ignore prob
		rmse = self.rmse(fx, y) 

		n = y.shape[0]  # will change for multiple outputs (y.shape[0]*y.shape[1])
		log_lhood = -n/2 * np.log(2 * math.pi * tausq) - (1/(2*tausq)) * np.sum(np.square(y - fx))
		return [log_lhood, fx, rmse]

	def prior_regression(self, sigma_squared, nu_1, nu_2, w, tausq): # for weights and biases and tausq
		h = self.topology[1]  # number hidden neurons
		d = self.topology[0]  # number input neurons
		part1 = -1 * ((d * h + h + 2) / 2) * np.log(sigma_squared)
		part2 = 1 / (2 * sigma_squared) * (sum(np.square(w)))
		log_loss = part1 - part2  - (1 + nu_1) * np.log(tausq) - (nu_2 / tausq)
		return log_loss


	def multinomial_loglikelihood(self, neuralnet, data, w):
		y = data[:, self.topology[0]]
		fx, prob = neuralnet.evaluate_proposal(data,w)
		acc= self.accuracy(fx,y)
		z = np.zeros((data.shape[0],self.topology[2]))
		lhood = 0
		for i in range(data.shape[0]):
			for j in range(self.topology[2]):
				if j == y[i]:
					z[i,j] = 1
				lhood += z[i,j]*np.log(prob[i,j])

		return [lhood, fx, acc]

	def prior_classification(self, sigma_squared, nu_1, nu_2, w): # for weights and biases only
		h = self.topology[1]  # number hidden neurons
		d = self.topology[0]  # number input neurons
		part1 = -1 * ((d * h + h + self.topology[2]+h*self.topology[2]) / 2) * np.log(sigma_squared)
		part2 = 1 / (2 * sigma_squared) * (sum(np.square(w)))
		log_loss = part1 - part2
		return log_loss


In [ ]:
'''
这段代码实现了贝叶斯神经网络中使用Metropolis-Hastings算法进行参数采样的过程，并使用Langevin梯度来提高采样效率。让我们逐步分解代码并理解其逻辑：

代码逐步解释
循环采样（for i in range(samples - 1)）：

代码在一个循环中执行多个采样步骤，循环次数为samples - 1。
随机变量lx用于控制是否使用Langevin梯度：

lx = np.random.uniform(0, 1, 1)生成一个0到1之间的随机数。
如果使用Langevin梯度（self.use_langevin_gradients is True）且lx小于self.l_prob，则应用Langevin动态来更新参数。
应用Langevin梯度更新参数：

w_gd = neuralnet.langevin_gradient(self.traindata, w.copy(), self.sgd_depth)：计算当前权重w在训练数据下的Langevin梯度。
w_proposal = np.random.normal(w_gd, step_w, w_size)：生成新的权重提议w_proposal，这是基于Langevin梯度的高斯分布。
w_prop_gd = neuralnet.langevin_gradient(self.traindata, w_proposal.copy(), self.sgd_depth)：计算提议权重的Langevin梯度。
diff_prop是一个对称性校正项，通过计算w和w_proposal在梯度方向上的差异来得到。
如果不使用Langevin梯度，diff_prop设置为0，且直接使用当前权重生成一个高斯分布的权重提议w_proposal。
计算似然和先验：

如果是回归问题，更新噪声参数eta_pro并计算相应的精度tau_pro。
self.loglikelihood(neuralnet, self.traindata, w_proposal, tau_pro)：计算提议权重在训练数据上的对数似然值。
self.loglikelihood(neuralnet, self.testdata, w_proposal, tau_pro)：计算提议权重在测试数据上的对数似然值（用于评估，不用于采样）。
self.prior(sigma_squared, nu_1, nu_2, w_proposal, tau_pro)：计算提议权重的先验概率。
计算差异项：

diff_prior = prior_prop - prior_current：计算先验的差异。
diff_likelihood = likelihood_proposal - likelihood：计算似然的差异。
Metropolis-Hastings接受概率：

mh_prob = min(1, math.exp(diff_likelihood + diff_prior + diff_prop))：计算Metropolis-Hastings接受概率。
如果对数值的和太大导致溢出，捕获OverflowError并设置接受概率为1，表示总是接受提议。
接受或拒绝提议：

生成随机数u = random.uniform(0, 1)。
如果u < mh_prob，接受提议权重：
更新当前的似然、先验、权重以及噪声参数（对于回归）。
naccept计数增加，用于跟踪接受的提议数目。
'''

		for i in range(samples - 1):

			lx = np.random.uniform(0,1,1)

			if (self.use_langevin_gradients is True) and (lx< self.l_prob):  
				w_gd = neuralnet.langevin_gradient(self.traindata, w.copy(), self.sgd_depth)  
				w_proposal = np.random.normal(w_gd, step_w, w_size)  
				w_prop_gd = neuralnet.langevin_gradient(self.traindata, w_proposal.copy(), self.sgd_depth) 
				#first = np.log(multivariate_normal.pdf(w , w_prop_gd , sigma_diagmat)) 
				#second = np.log(multivariate_normal.pdf(w_proposal , w_gd , sigma_diagmat)) 
				# this gives numerical instability - hence we give a simple implementation next that takes out log 

				wc_delta = (w- w_prop_gd) 
				wp_delta = (w_proposal - w_gd )

				sigma_sq = step_w

				first = -0.5 * np.sum(wc_delta  *  wc_delta  ) / sigma_sq  # this is wc_delta.T  *  wc_delta /sigma_sq
				second = -0.5 * np.sum(wp_delta * wp_delta ) / sigma_sq

				diff_prop =  first - second  
				langevin_count = langevin_count + 1

			else:
				diff_prop = 0
				w_proposal = np.random.normal(w, step_w, w_size)


			if self.prob == 'regression': 
				eta_pro = eta + np.random.normal(0, step_eta, 1)
				tau_pro = math.exp(eta_pro)
			else:# not used in case of classification
				eta_pro = 0
				tau_pro = 0


			[likelihood_proposal, pred_train, p_train] = self.loglikelihood(neuralnet, self.traindata, w_proposal,
																				tau_pro)
			[likelihood_ignore, pred_test, p_test] = self.loglikelihood(neuralnet, self.testdata, w_proposal,
																			tau_pro) 

			prior_prop = self.prior(sigma_squared, nu_1, nu_2, w_proposal,
											   tau_pro)  # takes care of the gradients


			diff_prior = prior_prop - prior_current

			diff_likelihood = likelihood_proposal - likelihood

			#mh_prob = min(1, math.exp(diff_likelihood + diff_prior + diff_prop))

			try:
				mh_prob = min(1, math.exp(diff_likelihood+diff_prior+ diff_prop))

			except OverflowError as e:
				mh_prob = 1


			u = random.uniform(0, 1)

			if u < mh_prob:
				# Update position 
				naccept += 1
				likelihood = likelihood_proposal
				prior_current = prior_prop
				w = w_proposal
				eta = eta_pro #only used for regression

In [ ]:
'''
这段代码使用了Python的NumPy、SciPy、Pandas、Matplotlib和Seaborn库来生成和可视化统计数据。具体步骤如下：

1. 导入必要的库
导入了常用的科学计算和数据可视化库，如NumPy、SciPy、Pandas、Matplotlib和Seaborn。
设置了Seaborn的样式为white，上下文为talk（适合展示）。
2. 生成随机数据
使用np.random.seed(123)来设置随机种子，使得结果可以复现。
使用data = np.random.randn(200)生成200个服从标准正态分布的随机数据。
3. 使用Seaborn绘制直方图
使用sns.distplot()绘制直方图，并且通过设置kde=True来添加核密度估计（KDE）曲线。
核密度估计是一种平滑数据分布的方式。
使用plt.savefig('histo_seaborn.png')保存图像为histo_seaborn.png。
使用plt.clf()清除当前图像，便于后续绘图。
4. 使用Matplotlib绘制直方图
使用plt.hist()绘制直方图，参数bins='auto'根据数据自动选择适合的分箱数。
设置颜色为#0504aa，不透明度为alpha=0.7，rwidth=0.85表示直方条的相对宽度。
添加网格和标签后，保存图像为histo_matplotlib.png，并使用plt.clf()清除当前图像。
5. 计算后验分布（使用贝叶斯更新）
定义了函数calc_posterior_analytical(data, x, mu_0, sigma_0)来计算后验分布。
假设观测数据data服从已知标准差的正态分布，使用贝叶斯定理来更新先验分布。
先验均值为mu_0，标准差为sigma_0。
计算后验均值mu_post和后验标准差sigma_post，返回相应的概率密度值。
使用calc_posterior_analytical(data, x, 0., 1.)计算在x范围内的后验概率密度，其中x = np.linspace(-1, 1, 50)为区间[-1, 1]上的50个点。
打印出计算的后验概率密度值，并绘制图像保存为histo_analytical.png。
6. 总结
该代码展示了如何使用Seaborn和Matplotlib绘制直方图，并将其保存到本地文件。
还展示了如何在贝叶斯推断中计算后验分布，使用简单的贝叶斯公式结合观测数据对均值mu进行更新，并绘制了后验分布。
总体来说，代码中包含了可视化数据、计算和展示贝叶斯后验的过程。
'''

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import norm

sns.set_style('white')
sns.set_context('talk')

np.random.seed(123)

data = np.random.randn(200)

#histogram using seaborn (sns)
ax = plt.subplot()
sns.distplot(data, kde=True, ax=ax) # smooth using KDE: https://en.wikipedia.org/wiki/Kernel_density_estimation
_ = ax.set(title='Histogram of observed data', xlabel='x', ylabel='frequency');
plt.tight_layout()
plt.savefig('histo_seaborn.png')
plt.clf()

#plot using matplotlib
# An "interface" to matplotlib.axes.Axes.hist() method
n, bins, patches = plt.hist(x=data, bins='auto', color='#0504aa', alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('x')
plt.ylabel('frequency')
plt.tight_layout()
plt.savefig('histo_matplotlib.png')
plt.clf()

def calc_posterior_analytical(data, x, mu_0, sigma_0):
    sigma = 1.
    n = len(data)
    mu_post = (mu_0 / sigma_0**2 + data.sum() / sigma**2) / (1. / sigma_0**2 + n / sigma**2)
    sigma_post = (1. / sigma_0**2 + n / sigma**2)**-1
    return norm(mu_post, np.sqrt(sigma_post)).pdf(x)

ax = plt.subplot()
x = np.linspace(-1, 1, 50)

posterior_analytical = calc_posterior_analytical(data, x, 0., 1.)
print(posterior_analytical)
ax.plot(x, posterior_analytical)
ax.set(xlabel='mu', ylabel='belief', title='Analytical posterior');
sns.despine()
plt.tight_layout()
plt.savefig('histo_analytical.png')
plt.clf()

 

In [ ]:
#source https://rdrr.io/cran/MCMCpack/man/MCMClogit.html
'''
这段代码使用 MCMCpack 包来进行逻辑回归的贝叶斯推断。

首先，加载 MCMCpack 包，并使用示例数据集 birthwt。代码中的每个示例使用 MCMClogit 函数对低出生体重(low)进行建模，预测变量包括母亲的年龄(age)、种族(race，因子化)和吸烟状态(smoke)。

默认先验是一个不适当的均匀分布（Improper Uniform Prior）。
使用多元正态分布作为先验，其中均值为0，方差为0.001。
使用用户定义的独立柯西分布先验，定义了一个 logpriorfun 函数，返回贝塔参数的柯西先验对数值。
定义带额外参数的独立柯西分布先验，包括位置(location)和尺度(scale)参数。
每个示例中，代码拟合模型(MCMClogit)，并用 plot 和 summary 函数来检查模型的后验分布和总结结果。
'''

library(MCMCpack)

   ## Not run: 
   ## default improper uniform prior
   data(birthwt)
   posterior <- MCMClogit(low~age+as.factor(race)+smoke, data=birthwt)
   plot(posterior)
   summary(posterior)


   ## multivariate normal prior
   data(birthwt)
   posterior <- MCMClogit(low~age+as.factor(race)+smoke, b0=0, B0=.001,
                          data=birthwt)
   plot(posterior)
   summary(posterior)


   ## user-defined independent Cauchy prior
   logpriorfun <- function(beta){
     sum(dcauchy(beta, log=TRUE))
   }

   posterior <- MCMClogit(low~age+as.factor(race)+smoke,
                          data=birthwt, user.prior.density=logpriorfun,
                          logfun=TRUE)
   plot(posterior)
   summary(posterior)


   ## user-defined independent Cauchy prior with additional args
   logpriorfun <- function(beta, location, scale){
     sum(dcauchy(beta, location, scale, log=TRUE))
   }

   posterior <- MCMClogit(low~age+as.factor(race)+smoke,
                          data=birthwt, user.prior.density=logpriorfun,
                          logfun=TRUE, location=0, scale=10)
   plot(posterior)
   summary(posterior)


   
## End(Not run)


# Week 7

In [ ]:
'''
导入必要的库：

numpy 和 pandas 用于数据处理。
eps 代表一个很小的浮点值，用于防止数值为零的除法错误。
log2 用于计算以 2 为底的对数（熵的定义需要用到对数）。
定义 find_entropy 函数：

find_entropy(df) 计算传入数据帧 df 的熵。
Class = df.keys()[-1] 获取数据帧中最后一列的名称，表示目标变量（在这里是“play”列）。
values = df[Class].unique() 获取目标变量中的所有唯一值（在这里是 "yes" 和 "no"）。
遍历 values，计算每个唯一值的频率（fraction），并使用熵公式 -fraction * log2(fraction) 计算熵值。
最终返回目标列的总熵。
创建数据集：

使用不同的天气属性（outlook、temp、humidity、windy）和目标变量 play 来构建数据集。
这些属性用逗号分隔的字符串表示，通过 .split(',') 转化为列表。
将所有属性组合为字典，使用 pd.DataFrame() 将其转换为数据帧 df。
打印数据帧和熵：

print(df) 打印数据集以查看数据的结构。
print(find_entropy(df)) 调用熵计算函数，并打印计算得到的熵。
'''
#source: https://medium.com/@lope.ai/decision-trees-from-scratch-using-id3-python-coding-it-up-6b79e3458de4
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log


def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    print(df.keys(), ' list of attributes')
    print(Class, ' is Class')
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
        print(df[Class].value_counts()[value], fraction, entropy, value)
    return entropy


outlook = 'overcast,overcast,overcast,overcast,rainy,rainy,rainy,rainy,rainy,sunny,sunny,sunny,sunny,sunny'.split(',')
temp = 'hot,cool,mild,hot,mild,cool,cool,mild,mild,hot,hot,mild,cool,mild'.split(',')
humidity = 'high,normal,high,normal,high,normal,normal,normal,high,high,high,high,normal,normal'.split(',')
windy = 'FALSE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE'.split(',')
play = 'yes,yes,yes,yes,yes,yes,no,yes,no,no,no,no,yes,yes'.split(',')

dataset ={'outlook':outlook,'temp':temp,'humidity':humidity,'windy':windy,'play':play}
df = pd.DataFrame(dataset,columns=['outlook','temp','humidity','windy','play'])

print(df)
print(find_entropy(df))

In [ ]:
'''

这段代码实现了ID3算法，从头开始构建决策树，找出每个属性的最佳划分方式，下面我来逐步解释：

导入库和初始化：

导入 numpy 和 pandas 以用于数值运算和数据处理。
eps 是一个非常小的值，用于防止数值为0时导致的除零错误。
log2 计算以2为底的对数，用于计算熵。
find_entropy(df) 函数：

计算整个数据集的熵。
首先获取目标变量（在这里是 'play' 列）的名称，然后对目标变量中每个唯一值计算其频率。
熵是通过 -fraction * log2(fraction) 的公式计算的，用来衡量目标变量的纯度。
find_entropy_attribute(df, attribute) 函数：

计算给定属性的条件熵，用于评估在某个属性的基础上划分数据集后的混乱程度。
首先，获取目标变量（如 'play' 列）和属性的所有唯一值。
对于每个属性值，计算该属性值下目标变量的熵。
通过对所有属性值计算并求和，得到整个属性的条件熵。
find_winner(df) 函数：

找到信息增益最高的属性，这就是在当前节点上最佳的划分属性。
信息增益的计算方式是通过目标变量的总熵减去给定属性的条件熵。
将所有属性的信息增益放入列表中，使用 np.argmax(IG) 找到信息增益最大的属性并返回。
创建数据集：

通过字符串列表创建数据集，包括 outlook（天气情况）、temp（温度）、humidity（湿度）、windy（是否有风）、play（是否适合玩耍）。
使用 pd.DataFrame() 将这些列表组合成一个数据帧 df。
找出最佳属性：

调用 find_winner(df) 函数获取信息增益最大的属性，即用于划分数据集的节点属性。
打印找到的节点属性（如 outlook）。
获取该属性的所有唯一值：

使用 np.unique(df[node]) 获取这个属性的所有可能取值。
'''
#source: https://medium.com/@lope.ai/decision-trees-from-scratch-using-id3-python-coding-it-up-6b79e3458de4
import numpy as np
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log


def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
        #print( entropy, value)
    return entropy

def find_entropy_attribute(df,attribute):
  Class = df.keys()[-1]   #To make the code generic, changing target variable class name
  target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
  variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
  entropy2 = 0
  for variable in variables:
      entropy = 0
      for target_variable in target_variables:
          num = len(df[attribute][df[attribute]==variable][df[Class] ==target_variable])
          den = len(df[attribute][df[attribute]==variable])
          fraction = num/(den+eps)
          entropy += -fraction*log(fraction+eps)
      fraction2 = den/len(df)
      entropy2 += -fraction2*entropy

  return abs(entropy2)


def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        Entropy_att.append(find_entropy_attribute(df,key))
        gain = find_entropy(df)-find_entropy_attribute(df,key)
        print(find_entropy(df), find_entropy_attribute(df,key), gain, key, ' gain, key')
        IG.append(gain)
        print(key, ' key')
    return df.keys()[:-1][np.argmax(IG)]
  


outlook = 'overcast,overcast,overcast,overcast,rainy,rainy,rainy,rainy,rainy,sunny,sunny,sunny,sunny,sunny'.split(',')
temp = 'hot,cool,mild,hot,mild,cool,cool,mild,mild,hot,hot,mild,cool,mild'.split(',')
humidity = 'high,normal,high,normal,high,normal,normal,normal,high,high,high,high,normal,normal'.split(',')
windy = 'FALSE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE'.split(',')
play = 'yes,yes,yes,yes,yes,yes,no,yes,no,no,no,no,yes,yes'.split(',')

dataset ={'outlook':outlook,'temp':temp,'humidity':humidity,'windy':windy,'play':play}
df = pd.DataFrame(dataset,columns=['outlook','temp','humidity','windy','play'])


#Get attribute with maximum information gain
node = find_winner(df)
print(node, ' node')

#Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
attValue = np.unique(df[node])
print(attValue, ' attValue')


In [ ]:
'''
这段代码实现了ID3算法来从头构建决策树，用于对数据进行分类。它选择信息增益最高的属性来构建决策树的节点，并通过递归方式来不断划分数据，直到数据集完全纯净为止。下面是代码的详细解释：

导入库和初始化
导入 numpy 和 pandas，用于数值运算和数据处理。
eps 是一个很小的数值，用于避免除以零的情况。
使用 log2 来计算熵。
1. find_entropy(df) 函数
功能：计算目标变量（例如 play 列）的熵，用于衡量数据集的混乱程度。
步骤：
获取目标变量（最后一列）的名称。
计算每个类别的频率，并计算熵（公式为 -fraction * log2(fraction)）。
2. find_entropy_attribute(df, attribute) 函数
功能：计算给定属性的条件熵，用于评估数据集根据该属性划分后的纯净度。
步骤：
获取目标变量和给定属性的所有唯一值。
对于每个属性值，计算目标变量的各类频率，并计算熵。
最后累加各个属性值对应的熵，得到整个属性的条件熵。
3. find_winner(df) 函数
功能：找到信息增益最大的属性，即用来划分数据集的最优属性。
步骤：
计算数据集的总熵。
遍历数据帧中的每个属性，计算其条件熵和信息增益。
找到信息增益最高的属性并返回其名称。
4. get_subtable(df, node, value) 函数
功能：根据节点属性值获取对应的数据子集。
步骤：
使用条件筛选，返回包含该属性值的数据子集，并重新索引。
5. buildTree(df, tree=None) 函数
功能：递归地构建决策树。
步骤：
首先，找到信息增益最大的属性，即最佳划分属性（node）。
获取该属性的所有唯一值，创建一个空字典用于构建树。
对于每个属性值，获取对应的子数据集（subtable），并检查它的纯净性：
如果子数据集纯净（即目标变量只有一个唯一值），将该值作为树的叶节点。
如果不纯净，递归调用 buildTree 来进一步划分。
最后返回构建完成的树。
数据集
创建了包含 outlook（天气情况）、temp（温度）、humidity（湿度）、windy（有无风）和 play（是否适合玩耍）的数据集。
使用这些属性构建一个数据帧 df，并打印出来查看其结构。
构建决策树
使用 buildTree(df) 函数构建决策树，并将结果存储在变量 tree 中。
打印树的结构。
示例输出说明
数据集：首先打印出整个数据集的结构。
节点选择：每次递归构建树时，会打印出选择的节点（最佳属性）及其属性值。
子集纯净性：对于每个子集，打印其目标变量的唯一值。
决策树结构：最后打印构建完成的树。
'''
#source: https://medium.com/@lope.ai/decision-trees-from-scratch-using-id3-python-coding-it-up-6b79e3458de4
import numpy as np
import pandas as pd
eps = np.finfo(float).eps # The difference between 1.0 and the next smallest representable float larger than 1.0. For example, for 64-bit binary floats in the IEEE-754 standard, eps = 2**-52, approximately 2.22e-16.
from numpy import log2 as log


def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*np.log2(fraction)
        #print( entropy, value)
    return entropy

def find_entropy_attribute(df,attribute):
  Class = df.keys()[-1]   #To make the code generic, changing target variable class name
  target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
  variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
  sum_entropy = 0
  
  for variable in variables:
      entropy = 0
      
      for target_variable in target_variables:
          num = len(df[attribute] [df[attribute] == variable] [df[Class] == target_variable]) # 'num' for numerator ... Count how many items in df[attribute] for each variable and target variable combination
          den = len(df[attribute] [df[attribute] == variable]) # 'den' for denominator ... Count how many items in df[attribute] for each variable
          fraction_target = num / (den + eps) # For each variable in df[attribute], the proportion where the target variable is a specific result ... Seems that eps is added to avoid dividing by zero
          entropy += -fraction_target * log(fraction_target + eps) # Adds up entropy calcs for each target variable within the attribute variable
          
      fraction_attribute = den / len(df) 
      sum_entropy += fraction_attribute * entropy # Once the entropies for the each target within the attribute variable are calculated they are multiplied by the fraction that the attribute represents of all attribute combinations

  return sum_entropy # for some reason this code added -fraction_attribute to sum_entropy then got the abs of the result ... not sure why since fractions and entropy are always positive


def find_winner(df):
    Entropy_att = []
    IG = []
    
    overall_entropy = find_entropy(df)
    #print('Overall entropy: ', overall_entropy)
    
    for key in df.keys()[:-1]:
        
        attribute_entropy = find_entropy_attribute(df,key)
        Entropy_att.append(attribute_entropy) # recorded in a list but not used
        
        gain = overall_entropy - attribute_entropy
        IG.append(gain) # used to index largest result
        
        #print('Attribute: ', key) 
        #print('Entropy: ', attribute_entropy, '  Gain: ', gain) 
        
    return df.keys()[:-1][np.argmax(IG)] # returns the key (attribute name) indexed to have the largest gain result
  

def get_subtable(df, node, value):
  return df[df[node] == value].reset_index(drop=True)


def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    #Here we build our decision tree

    #Get attribute with maximum information gain
    node = find_winner(df)    
    
    #Get distinct values of that attribute e.g Salary is node and Low, Med and High are values
    attValues = np.unique(df[node])
    
    print('Winning attribute node: ', node, attValues)
    
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree = {}
        tree[node] = {}
    
    #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 
    
    for value in attValues:
        
        subtable = get_subtable(df, node, value)
        print('\nSubtable: \n',subtable)
        
        class_values = subtable[Class].unique()
        print('Values:', class_values)
        
        #clValue, counts = np.unique(subtable, return_counts=True)    # could be a bug here                    
        #print('clValue: ',clValue)
        #print('counts: ',counts)
        
        if len(class_values) == 1:#Checking purity of subset #values instead of counts
            tree[node][value] = class_values[0]    #values instead of clValue                                               
        else: 
            #there is a bug here   (need to compare with code here: https://www.python-course.eu/Decision_Trees.php)    
            tree[node][value] = buildTree(subtable) #Calling the function recursively 
    
                   
    return tree


outlook = 'overcast,overcast,overcast,overcast,rainy,rainy,rainy,rainy,rainy,sunny,sunny,sunny,sunny,sunny'.split(',')
temp = 'hot,cool,mild,hot,mild,cool,cool,mild,mild,hot,hot,mild,cool,mild'.split(',')
humidity = 'high,normal,high,normal,high,normal,normal,normal,high,high,high,high,normal,normal'.split(',')
windy = 'FALSE,TRUE,TRUE,FALSE,FALSE,FALSE,TRUE,FALSE,TRUE,FALSE,TRUE,FALSE,FALSE,TRUE'.split(',')
play = 'yes,yes,yes,yes,yes,yes,no,yes,no,no,no,no,yes,yes'.split(',')

dataset = {'outlook':outlook,'temp':temp,'humidity':humidity,'windy':windy,'play':play}
df = pd.DataFrame(dataset,columns=['outlook','temp','humidity','windy','play'])
print(df)

tree = buildTree(df)
print(tree)

# Week 8

In [ ]:
'''
这段代码实现了一个基于 AdaBoost 算法的分类器，以下是对各部分的解释：

导入库和定义辅助函数：

numpy 用于数值计算。
DecisionTreeClassifier 是一个单层决策树（即弱分类器）。
AdaBoostClassifier 是一个集成学习库实现，虽然未直接使用它。
辅助函数 I(flag) 和 sign(x) 分别用于布尔转换和符号判断。
自定义 AdaBoost 类：

__init__() 方法：初始化 n_estimators，即弱分类器数量，同时创建一个存储这些分类器的列表。
fit() 方法：训练模型。
将输入数据 X 转换为浮点类型，初始化样本权重 w。
使用多个弱分类器（DecisionTreeClassifier），逐次进行训练，并根据分类误差来更新样本权重，以便让后续分类器更关注难以分类的样本。
计算每个弱分类器的权重 AlphaM，并更新样本权重 w。
将弱分类器及其权重存储到模型列表中。
predict() 方法：预测新样本。
利用所有弱分类器的加权预测结果，汇总为最终的分类结果。
测试模型：

使用 make_classification 生成测试数据。
标签转换：将 y 中为 0 的样本标签转换为 -1，以符合 AdaBoost 算法的要求。
实例化并训练自定义的 AdaBoost 模型。
预测数据，并计算模型的性能和混淆矩阵。
'''

import numpy as np

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import AdaBoostClassifier

def I(flag):
    return 1 if flag else 0

def sign(x):
    return abs(x)/x if x!=0 else 1       

class AdaBoost:
    
    def __init__(self,n_estimators=50):
        self.n_estimators = n_estimators
        self.models = [None]*n_estimators
        
    def fit(self,X,y):
        
        X = np.float64(X)
        N = len(y)
        w = np.array([1/N for i in range(N)])
        
        for m in range(self.n_estimators):
            
            Gm = DecisionTreeClassifier(max_depth=1).fit(X,y,sample_weight=w).predict
                        
            errM = sum([w[i]*I(y[i]!=Gm(X[i].reshape(1,-1))) for i in range(N)])/sum(w)
 
            AlphaM = np.log((1-errM)/errM)
            
            w = [w[i]*np.exp(AlphaM*I(y[i]!=Gm(X[i].reshape(1,-1)))) for i in range(N)] 
            
            
            self.models[m] = (AlphaM,Gm)

    def predict(self,X):
        
        y = 0
        for m in range(self.n_estimators):
            AlphaM,Gm = self.models[m]
            y += AlphaM*Gm(X)
        signA = np.vectorize(sign)
        y = np.where(signA(y)==-1,-1,1)
        return y

from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix as CM

x,y = make_classification(n_samples=217)
'''
As for our implementaion of AdaBoost 
y needs to be in {-1,1}
'''
y = np.where(y==0,-1,1)

clf = AdaBoost(n_estimators=5) # try 5 10 50 and press Run over and over again
clf.fit(x,y)
y_pred = clf.predict(x)


print("Performance:",100*sum(y_pred==y)/len(y))
print("Confusion Matrix:",CM(y,y_pred))

In [ ]:
'''

这段代码通过手动实现了一种类似于梯度提升（Gradient Boosting）的过程，用多个决策树来拟合数据。这是对梯度提升树的基本原理的演示。以下是详细的解释：

生成数据：

使用 numpy 生成随机数（设置随机种子为 42 以确保结果可复现）。
X 是一个包含 100 个样本的特征，每个值在 -0.5 到 0.5 之间。
y 是目标值，它的生成是基于 3 * X^2 的非线性函数，加上了一点随机噪声。
拟合多个回归树：

第一个决策树 tree_reg1 使用 max_depth=2 进行拟合，拟合数据集 (X, y)。
残差计算：计算第一次拟合后的残差，即 y2 = y - tree_reg1.predict(X)，它代表原始目标值和模型预测之间的差异。
第二个决策树 tree_reg2 用来拟合残差 y2，同样使用最大深度为 2。
再次残差计算：计算第二次拟合后的残差 y3 = y2 - tree_reg2.predict(X)，代表第二个模型拟合后的差异。
第三个决策树 tree_reg3 用来拟合第二次残差 y3。
预测新样本：

定义一个新的样本点 X_new = [[0.8]]。
使用每个回归树分别对 X_new 进行预测，并将预测值累加起来，即 y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))。
累加的过程相当于各个树的预测值进行加权和，从而得到最终的预测结果，这种方法类似于梯度提升。
打印结果：

打印了每个回归树对 X_new 的预测结果。
打印了最终的梯度提升预测结果。

'''

import numpy as np
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X, y2)

y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X, y3)

X_new = np.array([[0.8]])

y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))


print('tree_reg1.predict : ', tree_reg1.predict(X_new) )
print('tree_reg2.predict : ', tree_reg2.predict(X_new) )
print('tree_reg3.predict : ', tree_reg3.predict(X_new) )
print('---------- ------------')
print('Gradient Boosting: ', y_pred )
print('---------- ------------')


In [ ]:
'''
这段代码比较了多种单一模型与一个堆叠（stacking）集成模型的分类性能。以下是代码的逐步解释：

导入库：导入了多个必要的库来处理数据集、评估模型、以及实现不同的分类算法。

数据集生成 (get_dataset)：使用 make_classification 生成一个包含 1000 个样本、20 个特征的合成分类数据集，其中 15 个特征是信息性的，5 个是冗余特征。

定义堆叠模型 (get_stacking)：

定义了 5 个基础模型（逻辑回归、KNN、决策树、SVM、朴素贝叶斯）作为底层学习器。
使用逻辑回归作为元学习器，将基础模型的输出进行学习和融合。
最终定义一个堆叠集成模型 StackingClassifier，通过交叉验证 (cv=5) 进一步增强性能。
获取所有要评估的模型 (get_models)：定义了 5 个单独的分类器和一个堆叠模型，总共 6 个模型，存储在一个字典中。

模型评估 (evaluate_model)：

使用 RepeatedStratifiedKFold 进行 10 折交叉验证（重复 3 次），确保模型评估更加稳健。
计算每个模型在交叉验证中的准确率，并返回分数列表。
评估和比较模型：

使用定义的数据集和模型列表，对每个模型进行交叉验证评估，并计算平均准确率和标准差。
输出每个模型的评估结果，显示模型的名称、平均准确率、以及标准差。
可视化比较：

使用 boxplot 绘制不同模型的准确率分布图，以便于比较模型性能差异。
将图表保存为 stacked.png。
'''

# compare ensemble to each baseline classifier
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot

# get the dataset
def get_dataset():
	X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=1)
	return X, y

# get a stacking ensemble of models
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('lr', LogisticRegression()))
	level0.append(('knn', KNeighborsClassifier()))
	level0.append(('cart', DecisionTreeClassifier()))
	level0.append(('svm', SVC()))
	level0.append(('bayes', GaussianNB()))
	# define meta learner model
	level1 = LogisticRegression()
	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)
	return model

# get a list of models to evaluate
def get_models():
	models = dict()
	models['lr'] = LogisticRegression()
	models['knn'] = KNeighborsClassifier()
	models['cart'] = DecisionTreeClassifier()
	models['svm'] = SVC()
	models['bayes'] = GaussianNB()
	models['stacking'] = get_stacking()
	return models

# evaluate a give model using cross-validation
def evaluate_model(model, X, y):
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
	return scores

# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X, y)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.savefig('stacked.png')

# Week 9

In [ ]:
# SVD数学计算代码

from numpy import array
from numpy import diag
from numpy import zeros
from scipy.linalg import svd
# define a matrix
A = array([
	[1,2,3,4,5,6,7,8,9,10],
	[11,12,13,14,15,16,17,18,19,20],
	[21,22,23,24,25,26,27,28,29,30]])
print(A)
# Singular-value decomposition
# A = USV (S为sigma,但是表示为一个向量而非矩阵，包含了sigma矩阵的全部对角元素)
U, s, VT = svd(A)

# 使用对角元素向量s构建sigma矩阵
# create m x n Sigma matrix
Sigma = zeros((A.shape[0], A.shape[1]))
# populate Sigma with n x n diagonal matrix
Sigma[:A.shape[0], :A.shape[0]] = diag(s)

# 特征选择，选择sigma的前两列，V的前两行
# select
n_elements = 2
Sigma = Sigma[:, :n_elements]
VT = VT[:n_elements, :]

# 点乘，B是原矩阵A的一个低秩近似
# reconstruct
B = U.dot(Sigma.dot(VT))
print(B)

# SVD分解结果为T，有两种计算方法，结果一致
# transform
T = U.dot(Sigma)
print(T)
T = A.dot(VT.T)
print(T)

#Running the example first prints the defined matrix then the reconstructed
# approximation, followed by two equivalent transforms of the original matrix.

print(' Truncated SVD')
#Truncated SVD

#The TruncatedSVD class can be created in which you must specify the number
# of desirable features or components to select, e.g. 2. Once created, 
#you can fit the transform (e.g. calculate V^Tk) by calling the fit() 
#function, then apply it to the original matrix by calling the transform() 
#function. The result is the transform of A called T above.

from sklearn.decomposition import TruncatedSVD
# define array
A = array([
	[1,2,3,4,5,6,7,8,9,10],
	[11,12,13,14,15,16,17,18,19,20],
	[21,22,23,24,25,26,27,28,29,30]])
print(A)
# svd
svd = TruncatedSVD(n_components=2)
svd.fit(A)
result = svd.transform(A)
print(result)

In [ ]:
# 同理，只是这段代码用了np
import numpy as np
U, s, Vt = np.linalg.svd(A)
W2 = Vt.T[:, :2]
X2D = A.dot(W2)

In [ ]:
# 增量PCA

# Authors: Kyle Kastner
# License: BSD 3 clause

import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
from sklearn.decomposition import PCA, IncrementalPCA

# 加载数据
iris = load_iris()
X = iris.data
y = iris.target

n_components = 2
ipca = IncrementalPCA(n_components=n_components, batch_size=10)
X_ipca = ipca.fit_transform(X)

pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X)

print(X_ipca, ' X_ipca')

colors = ['navy', 'turquoise', 'darkorange']

for X_transformed, title in [(X_ipca, "Incremental PCA"), (X_pca, "PCA")]:
    plt.figure(figsize=(8, 8))
    for color, i, target_name in zip(colors, [0, 1, 2], iris.target_names):
        plt.scatter(X_transformed[y == i, 0], X_transformed[y == i, 1],
                    color=color, lw=2, label=target_name)

    if True or "Incremental" in title:
        err = np.abs(np.abs(X_pca) - np.abs(X_ipca)).mean()
        plt.title(title + " of iris dataset\nMean absolute unsigned error "
                  "%.6f" % err)
    else:
        plt.title(title + " of iris dataset")
    plt.legend(loc="best", shadow=False, scatterpoints=1)
    plt.axis([-4, 4, -1.5, 1.5])

plt.savefig('ipca.png')


In [ ]:
#K-Means 聚类算法

import matplotlib.pyplot as plt
from matplotlib import style
#style.use('ggplot')
import numpy as np


class K_Means:
    def __init__(self, k=3, tol=0.01, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter

        self.centroids = {}

    def fit(self,data):


        for i in range(self.k):
            self.centroids[i] = data[i]

        for i in range(self.max_iter):
            print(i, self.centroids, ' centroids')
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []
            
            print(self.classifications, i, ' * ')

            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids] # check distance of every data point to every centroid
                print(distances, featureset, ' dist - featureset')
                classification = distances.index(min(distances)) # get index of centroid that best suits the data point
                self.classifications[classification].append(featureset) # assign the centroid id to the data point
            
            print(self.classifications, i, ' + ')

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids: # update centroid position
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    print(np.sum((current_centroid-original_centroid)/original_centroid*100.0))
                    optimized = False

            if optimized:
                break
        return self.centroids

    def predict(self,data):
        distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        return classification

#main

X = np.array([[1, 2],
              [1.5, 1.8],
              [5, 8 ],
              [8, 8],
              [1, 0.6],
              [9,11],
              [1,3],
              [8,9],
              [0,3],
              [5,4],
              [6,4],])

plt.scatter(X[:,0], X[:,1], s=150)
plt.savefig('data.png')
print(X, ' data')

colors = 10*["g","r","c","b","k"]
clf = K_Means()
x = clf.fit(X)
print(x, 'final centroids')

for centroid in clf.centroids:
    plt.scatter(clf.centroids[centroid][0], clf.centroids[centroid][1],
                marker="o", color="k", s=150, linewidths=5)

for classification in clf.classifications:
    color = colors[classification]
    for featureset in clf.classifications[classification]:
        plt.scatter(featureset[0], featureset[1], marker="x", color=color, s=150, linewidths=5)

# in case we wish to test if data belongs to a cluster

##unknowns = np.array([[1,3],
##                     [8,9],
##                     [0,3],
##                     [5,4],
##                     [6,4],])
##
##for unknown in unknowns:
##    classification = clf.predict(unknown)
##    plt.scatter(unknown[0], unknown[1], marker="*", color=colors[classification], s=150, linewidths=5)
##

plt.savefig('kmeans.png')

In [ ]:
# 软聚类代码

# Common imports
import numpy as np
import os
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
# Let's start by generating some blobs ---
blob_centers = np.array(
    [[ 0.2,  2.3],
     [-1.5 ,  2.3],
     [-2.8,  1.8],
     [-2.8,  2.8],
     [-2.8,  1.3]])
blob_std = np.array([0.4, 0.3, 0.1, 0.1, 0.1])
X, y = make_blobs(n_samples=2000, centers=blob_centers,
                  cluster_std=blob_std, random_state=7)
# ------- end of generating blobs ----- 
k = 5
kmeans = KMeans(n_clusters=k)
y_pred = kmeans.fit_predict(X)

# The following assigns new instances to the cluster whose centroid is closest
X_new = np.array([[0, 2], [3, 2], [-3, 3], [-3, 2.5]])

print('kmeans.transform(X_new):')
print(kmeans.transform(X_new))



In [ ]:
# 分层聚类算法

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
#The next step is to import or create the dataset. In this example, we'll use the following example data:

X = np.array([[5,3],
    [10,15],
    [15,12],
    [24,10],
    [30,30],
    [85,70],
    [71,80],
    [60,78],
    [70,55],
    [80,91],])

from sklearn.cluster import AgglomerativeClustering
cluster = AgglomerativeClustering(n_clusters=2, linkage='ward')
cluster.fit_predict(X) 
print(cluster.labels_)

plt.scatter(X[:,0],X[:,1], c=cluster.labels_, cmap='rainbow')
plt.savefig('clusters.png')



In [ ]:
# DBSCAN聚类
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons
import numpy as np

X, y = make_moons(n_samples=1000, noise=0.05)
print(X)
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)
print('dbscan.labels_[:10]: ', dbscan.labels_[:10])
print('np.unique(dbscan.labels_): ', np.unique(dbscan.labels_))


In [ ]:
# DBSCAN 聚类
import numpy as np

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler


# #############################################################################
# Generate sample data
# 生成样本数据
centers = [[1, 1], [-1, -1], [1, -1]]
X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
                            random_state=0)

X = StandardScaler().fit_transform(X)

# #############################################################################
# Compute DBSCAN 计算聚类
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present. 计算簇数量和噪声点数量
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

# Homogeneity：同质性，所有簇只包含单一类别的样本的程度。
# Completeness：完整性，所有同一类别的样本被分配到同一簇的程度。
# V-measure：同质性和完整性的调和平均值。
# Adjusted Rand Index (ARI)：调整后的兰德指数，衡量聚类与真实标签的一致性。
# Adjusted Mutual Information (AMI)：调整后的互信息，衡量聚类结果和真实标签的信息共享程度。
# Silhouette Coefficient：轮廓系数，衡量聚类结果的紧密性和分离性，值越接近 1，聚类效果越好。


print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
print("Adjusted Rand Index: %0.3f"
      % metrics.adjusted_rand_score(labels_true, labels))
print("Adjusted Mutual Information: %0.3f"
      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

# #############################################################################
# Plot result
import matplotlib.pyplot as plt

# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.savefig('DBC.png')

# Week 10

In [ ]:
'''这段代码描述了时间序列模型中应用的“反向传播时间”算法（Back Propagation Through Time, BPTT），用于训练递归神经网络（RNN）。算法通过展开网络来处理序列中的依赖关系，并根据误差对权重进行更新。具体步骤如下：

将网络展开为包含 k 个实例的网络（即展开 k 个时间步）。
反复执行以下步骤直到满足停止条件：
初始化上下文向量 x 为零向量（表示当前上下文）。
遍历训练序列，从时间步 t 开始，对所有时间步进行迭代。
将展开网络的输入设置为当前上下文 x 和从时间 t 到 t+k-1 的输入数据 a[t], a[t+1], ..., a[t+k-1]。
前向传播计算网络的输出 p，得到网络在时间步 t+k 的预测值。
计算误差 e，即目标值 y[t+k] 与预测值 p 的差。
通过整个展开的网络反向传播误差 e，计算每个时间步的误差对权重的影响。
将网络展开后的每个实例的权重变化加和起来。
更新模型的所有权重。
使用当前时间步的输入 a[t] 和上下文 x 计算新的上下文，用于下一个时间步的输入。'''

Back_Propagation_Through_Time(a, y)   // a[t] is the input at time t. y[t] is the output
    Unfold the network to contain k instances of f
    do until stopping criteria is met:
        x := the zero-magnitude vector // x is the current context
        for t from 0 to n − k do      // t is time. n is the length of the training sequence
            Set the network inputs to x, a[t], a[t+1], ..., a[t+k−1]
            p := forward-propagate the inputs over the whole unfolded network
            e := y[t+k] − p;           // error = target − prediction
            Back-propagate the error, e, back across the whole unfolded network
            Sum the weight changes in the k instances of f together.
            Update all the weights in f and g.
            x := f(x, a[t]);           // compute the context for the next time-step

In [ ]:
'''
这段代码实现了一个递归神经网络（RNN）的前向传播（ForwardPass）和反向传播时间算法（BPTT，Back Propagation Through Time）。以下是对代码的逐步解释：

前向传播函数 ForwardPass(self, sample, slide):

该函数用于处理输入样本的前向传播，计算每层的输出。
sample_time = sample[slide] 获取当前时间步的输入数据。
第一层（输入层到隐藏层）的前向传播：
将输入样本的特征值赋给 InlayerOutL0，保存在时间步 slide + 1 的输入中。
使用输入层的输出和权重矩阵 W1 计算每个隐藏单元的加权和（weightsum）。
使用上一时间步隐藏层状态 OutputSlideL1 和状态权重矩阵 StateW 计算当前状态贡献（StateWeightSum）。
计算总的输入加权和减去偏置 B1，并使用激活函数（sigmoid）计算隐藏层输出。
第二层（隐藏层到输出层）的前向传播：
计算隐藏层的输出与权重矩阵 W2 的加权和（weightsum），并使用激活函数 sigmoid 计算最终输出。
反向传播时间函数 BPTT(self):

该函数实现了训练递归神经网络的“反向传播时间”算法。
对于每个样本，初始化隐藏层的初始状态 StateOut，并准备误差存储 (ErL1) 以及前向传播输出存储 (OutputSlideL1 和 OutputSlideL2)。
对于每个时间步，调用 ForwardPass 计算前向传播的输出。
在完成整个序列的前向传播后，反向从最后时间步到第一个时间步，调用 BackwardPass 进行反向传播，以调整网络的权重。
总体上，这段代码实现了一个具有递归连接的神经网络，它利用前向传播来计算每个时间步的输出，并使用反向传播时间（BPTT）算法来学习网络的权重和偏置，以便使模型能够更好地拟合训练数据。在 ForwardPass 中，隐藏层的状态不仅依赖于当前输入，还依赖于前一时间步的状态，因此能够捕捉到时间上的依赖关系。而 BPTT 则通过逐时间步地反向传播误差来更新网络参数。
'''

   # [t1 (f1, f2, f3), t2 (f1, f2, f3), t3 (f1, f2, f3)] ->t4  (FNN) 

#RNN
   # [t1] - f1, f2, f3
   # [t2] - f1, f2, f3
   # [t3] - f1, f2, f3
   # [t4] - f1 


   def ForwardPass(self, sample,slide):
        sample_time = sample[slide]
        layer = 0 
        weightsum = 0.0
        StateWeightSum = 0.0
        forwardout=0.0 
        for row in range(0,self.Top[0]):
            self.InlayerOutL0[slide+1][row] = sample_time[row]

        for y in range(0, self.Top[1]):
            for x in range(0, self.Top[0]):
                weightsum += self.InlayerOutL0[slide+1][x] * self.W1[x,y]
           
            for x in range(0,self.Top[1]):
                StateWeightSum += self.OutputSlideL1[slide][x] * self.StateW[x,y]
            forwardout = (weightsum + StateWeightSum) - self.B1[y]
          
            self.OutputSlideL1[slide+1][y] = self.sigmoid(forwardout)            
            weightsum=0
            StateWeightSum=0

        layer = 1 #   hidden layer to output
        weightsum = 0.0
        #print(self.out,end=' ')
        for y in range(0, self.Top[layer+1]):
            for x in range(0, self.Top[layer]):
                weightsum  +=   self.OutputSlideL1[slide+1][x] * self.W2[x,y]
                forwardout = (weightsum - self.B2[y])
            self.OutputSlideL2[slide+1][y] = self.sigmoid(forwardout) 
            weightsum = 0.0
            StateWeightSum=0.0       
   
   
   
   
    def BPTT(self):  
        for i,sample in enumerate(self.Train_x): 
            self.StateOut = np.ones(self.Top[1])
            self.ErL1 = np.zeros((len(sample)+1,self.Top[1])) # need to modify for multiple layers
            self.OutputSlideL1 = np.zeros((len(sample)+1,self.Top[1]))
            for x in range(0,self.Top[1]):
                self.OutputSlideL1[0][x] = self.StateOut[x]
            self.InlayerOutL0 = np.zeros((len(sample)+1,self.Top[0]))
            self.OutputSlideL2 = np.zeros((len(sample)+1,self.Top[2]))
            for slide in range(0,len(sample)):
                self.ForwardPass(sample,slide)
            for slide in range(len(sample),0,-1):
                self.BackwardPass(sample,self.learn_rate,self.Train_y[i-1],slide)


In [ ]:
'''
这段代码实现了一个简单的递归神经网络（RNN），以下是对代码的解释：

类 RecurrentNetwork 的定义:

__init__(self):
初始化了类的一些参数，包括：
hidden_state：初始隐藏状态矩阵为零，形状为 (3, 3)。
W_hh：用于隐藏状态到隐藏状态的权重矩阵，随机初始化。
W_xh：用于输入到隐藏状态的权重矩阵，随机初始化。
W_hy：用于隐藏状态到输出的权重矩阵，随机初始化。
Bh 和 By：分别是隐藏层和输出层的偏置，随机初始化。
这些权重和偏置用于对输入进行线性变换，以计算新的隐藏状态和最终输出。
前向传播函数 forward_prop(self, x):

该函数接收一个输入 x，然后计算新的隐藏状态和输出。
计算过程如下：
通过 np.dot(self.hidden_state, self.W_hh) 计算上一时间步隐藏状态与权重矩阵的点积。
通过 np.dot(x, self.W_xh) 计算输入向量与输入到隐藏层的权重矩阵的点积。
上述两项加上隐藏层的偏置 Bh，通过 np.tanh() 激活函数计算新的隐藏状态。
最后通过 self.W_hy.dot(self.hidden_state) + self.By 计算隐藏状态到输出层的输出。
运行代码部分:

初始化了一个输入向量 input_vector，大小为 (3, 3)，其元素均为 1。
然后创建了 RecurrentNetwork 的一个实例 silly_network。
使用 silly_network.forward_prop(input_vector) 三次调用前向传播函数，并打印输出。
每次调用时，前向传播的结果都会不同，因为递归神经网络中的隐藏状态是不断更新的，它依赖于之前的时间步，这使得相同的输入在不同的时间步产生不同的输出。

'''

import numpy as np


np.random.seed(0)
class RecurrentNetwork(object):
    """When we say W_hh, it means a weight matrix that accepts a hidden state and produce a new hidden state.
    Similarly, W_xh represents a weight matrix that accepts an input vector and produce a new hidden state. This
    notation can get messy as we get more variables later on with LSTM and I simplify the notation a little bit in
    LSTM notes.
    """
    def __init__(self):
        self.hidden_state = np.zeros((3, 3))
        self.W_hh = np.random.randn(3, 3)
        self.W_xh = np.random.randn(3, 3)
        self.W_hy = np.random.randn(3, 3)
        self.Bh = np.random.randn(3,)
        self.By = np.random.rand(3,)

    def forward_prop(self, x):
        # The order of which you do dot product is entirely up to you. The gradient updates will take care itself
        # as long as the matrix dimension matches up.
        self.hidden_state =         return self.W_hy.dot(self.hidden_state) + self.By
np.tanh(np.dot(self.hidden_state, self.W_hh) + np.dot(x, self.W_xh) + self.Bh)


input_vector = np.ones((3, 3))
print(input_vector, ' input vec')
silly_network = RecurrentNetwork()


# Notice that same input, but leads to different ouptut at every single time step.
print(silly_network.forward_prop(input_vector))
print(silly_network.forward_prop(input_vector))
print(silly_network.forward_prop(input_vector))

In [ ]:
'''
加载和准备数据：

使用Keras的imdb数据集加载电影评论，设置词汇表大小为5000个最常见单词。
评论数据以单词索引的形式加载，正面和负面评论分别标注为1和0。
提取评论的最大和最小长度，以了解数据的分布情况。
数据预处理：

使用单词到索引的映射将索引序列转换回评论文本，方便查看评论内容。
将所有评论填充或截断为固定长度500，以便输入到神经网络中进行训练。
构建情感分析模型：

创建一个顺序模型，包含以下层：
Embedding层：将单词索引映射为嵌入向量，大小为32，用于捕获单词之间的语义关系。
LSTM层：具有100个单元，用于提取评论中的时间序列特征。
Dense层：使用sigmoid激活函数，将LSTM的输出映射为一个概率值，用于二分类。
最后打印模型的结构信息。
'''

from keras.datasets import imdb
#Set the vocabulary size and load in training and test data.
vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print('---review with words---')
print([id2word.get(i, ' ') for i in X_train[6]])
print('---label---')
print(y_train[6])

print('Maximum review length: {}'.format(
len(max((X_train + X_test), key=len))))
#Maximum review length: 2697
print('Minimum review length: {}'.format(
len(min((X_test + X_test), key=len))))

from keras.preprocessing import sequence
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
embedding_size=32
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
print(model.summary())


In [ ]:
'''
库的导入：导入了numpy、Keras等库，用于数据处理、模型定义和训练。

函数定义：generate_seq：

这是一个生成文本序列的函数，它基于训练好的模型和种子文本生成新的单词。
in_text是开始的种子文本，通过循环将预测的单词逐步添加到in_text，生成一段指定长度的文本。
准备文本数据：

data是输入的文本，描述了“Jack and Jill”的故事。
通过Tokenizer将文本转换为整数序列，即将每个单词映射到一个唯一的整数值。
计算词汇表大小，以便定义嵌入层的输入维度。
生成训练数据：

创建序列用于训练，使用前两个单词预测下一个单词。
例如，对于句子“Jack and Jill went”，序列生成会是“Jack and Jill -> Jill went”这样的三元组，最后一个单词是目标。
填充序列：

由于文本的不同部分可能长度不同，通过pad_sequences对序列进行填充，使它们的长度一致，便于模型处理。
输入和输出的拆分：

输入（X）是句子中前两个单词的整数表示，输出（y）是预测的目标单词。
通过to_categorical将输出转换为独热编码，便于分类训练。
定义模型：

使用Keras定义了一个顺序模型（Sequential）。
第一个层是嵌入层（Embedding），用于将整数序列转换为稠密的向量表示。
接着是一个LSTM层，用于处理序列数据。
最后是一个全连接的输出层，用于预测下一个单词的概率分布。
编译和训练模型：

使用categorical_crossentropy损失函数和adam优化器来编译模型。
用输入X和输出y训练模型500次。
生成新文本：

使用generate_seq函数基于不同的种子文本生成新的单词。
'''

from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

# generate a sequence from a language model
def generate_seq(model, tokenizer, max_length, seed_text, n_words):
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# pre-pad sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=max_length, padding='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
	return in_text

# source text
data = """ Jack and Jill went up the hill\n
		To fetch a pail of water\n
		Jack fell down and broke his crown\n
		And Jill came tumbling after\n """
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]
# retrieve vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)
# encode 2 words -> 1 word
sequences = list()
for i in range(2, len(encoded)):
	sequence = encoded[i-2:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))
# pad sequences
max_length = max([len(seq) for seq in sequences])
sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
print('Max Sequence Length: %d' % max_length)
# split into input and output elements
sequences = array(sequences)
X, y = sequences[:,:-1],sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_length-1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=500, verbose=2)
# evaluate model
print(generate_seq(model, tokenizer, max_length-1, 'Jack and', 5))
print(generate_seq(model, tokenizer, max_length-1, 'And Jill', 3))
print(generate_seq(model, tokenizer, max_length-1, 'fell down', 5))
print(generate_seq(model, tokenizer, max_length-1, 'pail of', 5))

In [ ]:
'''
这段代码实现了一个简单的卷积神经网络（CNN）来对MNIST数据集进行分类。MNIST数据集是一个手写数字的数据集，包含从0到9的数字。以下是对代码的逐步解释：

1. 导入必要的库
使用了Keras库来构建和训练卷积神经网络。
导入了必要的模块和函数，例如卷积层、池化层、全连接层等。
2. 定义参数
num_classes：类别数量（10个，分别对应数字0-9）。
num_epochs：训练过程中遍历数据的次数，这里设为1。
img_rows, img_cols：输入图片的大小（28x28像素）。
3. 加载MNIST数据集
mnist.load_data() 返回训练和测试数据集。
x_train、y_train为训练数据及标签，x_test、y_test为测试数据及标签。
4. 数据预处理
根据后端（Theano或TensorFlow）的偏好，调整数据的形状。TensorFlow使用(rows, cols, channels)格式，Theano使用(channels, rows, cols)格式。
将像素值归一化到0-1之间（将像素值除以255），以便更快地训练模型。
将标签转换为独热编码（one-hot encoding）格式，例如数字“3”转换为[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]。
5. 打印样本数量
打印训练和测试样本的数量。
6. 构建卷积神经网络模型
构建了一个顺序模型（Sequential）。
卷积层（Conv2D）：使用32个3x3大小的卷积核，激活函数为ReLU，输入形状为(28, 28, 1)。
池化层（MaxPooling2D）：使用2x2大小的最大池化层来减少特征图的大小，从而加快计算并避免过拟合。
代码中有注释提到如果添加更多的池化层可能会进一步加快计算，但会导致准确度下降。
Flatten层：将特征图转换为一维向量，以便传递给全连接层。
全连接层（Dense）：128个神经元，激活函数为ReLU。
输出层（Dense）：输出大小为num_classes（即10），激活函数为softmax，用于多分类问题。
7. 编译模型
使用损失函数categorical_crossentropy，适用于多分类问题。
使用优化器sgd（随机梯度下降）。
指标为准确率。
8. 训练模型
使用model.fit()对训练数据进行训练，epochs=num_epochs表示训练数据遍历的次数。
使用validation_data在训练过程中评估模型的性能。
9. 评估模型
使用model.evaluate()在测试集上评估模型，返回测试损失和测试准确率。
打印出测试集上的损失和准确率。
10. 保存模型
保存模型结构：使用YAML格式将模型保存到文件model.yaml中。
保存模型权重：将模型的权重保存到文件model_weights.h5中。
'''
import keras
from keras import backend as K
from keras.datasets import mnist
from keras.models import Sequential
from keras.models import model_from_yaml
from keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten


# A "class" for each digit from 0-9
num_classes = 10
# How many times to run through the data while training
num_epochs = 1
# Input (image) dimensions
img_rows, img_cols = 28, 28

# mnist.load_data() returns 2 tuples split into training/testing
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Need to reshape data based on backend preferred image format (TF vs Theano)
if K.image_data_format() == 'channels_first':
	x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols).astype('float32')
	x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols).astype('float32')
	input_shape = (1, img_rows, img_cols)
else:
	x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1).astype('float32')
	x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1).astype('float32')
	input_shape = (img_rows, img_cols, 1)

# Normalize pixel values between 0 and 1 per color channel for easier training
x_train /= 255
x_test /= 255

# Convert class label values to one-hot vectors [0, 0, ..., 1, 0, ..., 0]
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Print out sample sizes
print("Training samples:", x_train.shape[0])
print("Test samples:", x_test.shape[0])

# Build model
model = Sequential()
# Convolutional Layer (input shape specified because its the first layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
# Pooling Layer (speeds up computation somewhat by decreasing data size and
# 				 allows for more local features to be learned)
model.add(MaxPooling2D((2, 2)))
# speeds up from 1min to 40 seconds but worse accuracy
#model.add(MaxPooling2D((2, 2)))
# doesn't speed up anymore and worse accuracy
#model.add(MaxPooling2D((2, 2)))

# Flattens data into one dimension for fully connected layer to follow
model.add(Flatten())
# Fully Connected Layer
model.add(Dense(128, activation='relu'))
# Output layer
model.add(Dense(num_classes, activation='softmax'))

# Choosing loss function and optimizer for training
model.compile(loss='categorical_crossentropy',
				optimizer='sgd',
				metrics=['accuracy'])

# Fit to training data (like sklearn classifiers)
model.fit(x_train, y_train, 
		  epochs=num_epochs, 
		  validation_data=(x_test, y_test))

# Save metrics of network like accuracy for observation
score = model.evaluate(x_test, y_test, verbose=0)

# Print out results
print("Test loss:", score[0])
print("Test accuracy: {:.2f}%".format(score[1]*100))

# Save model
model_yaml = model.to_yaml()
with open('model.yaml', 'w') as yaml_file:
	yaml_file.write(model_yaml)

# Save weights
model.save_weights('model_weights.h5')
print('Saved model and weights to disk.')